<a href="https://colab.research.google.com/github/atlas-github/20190731StarMediaGroup/blob/master/sales_forecasting/gbr_plus_purchasing_power.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime as date

# Import data

In [5]:
py_data = pd.read_parquet("df_sales_by_hour_py.parquet")
cy_data = pd.read_parquet("df_sales_by_hour_cy.parquet")

## OPR data

In [52]:
#2022 source: https://www.bnm.gov.my/monetary-stability/opr-decisions/-/tag/opr-2022
#2023 source: https://www.bnm.gov.my/monetary-stability/opr-decisions/-/tag/opr-2023
#2024 source: https://www.bnm.gov.my/monetary-stability/mpc-meetings

opr_data = {
    'date': ["2022-01-20", "2022-03-03", "2022-05-11", "2022-07-06", "2022-09-08", "2022-11-03",
             "2023-01-19", "2023-03-09", "2023-05-03", "2023-07-06", "2023-09-07", "2023-11-02",
             "2024-01-24", "2024-03-07", "2024-05-09", "2024-07-11", "2024-09-05", "2024-11-06"],
    'change_in_opr': [0, 0, 0.25, 0.25, 0.25, 0.25,
                      0, 0, 0.25, 0, 0, 0,
                      0, 0, 0, 0, 0, 0],
    'new_opr_level': [1.75, 1.75, 2.00, 2.25, 2.50, 2.75,
                      2.75, 2.75, 3.00, 3.00, 3.00, 3.00,
                      3.00, 3.00, 3.00, 3.00, 3.00, 3.00]
}

In [56]:
df_opr_data = pd.DataFrame(opr_data)
df_opr_data["date"] = pd.to_datetime(df_opr_data["date"])

In [58]:
df_opr_data

,date,change_in_opr,new_opr_level
0,2022-01-20,0.00,1.75
1,2022-03-03,0.00,1.75
2,2022-05-11,0.25,2.00
3,2022-07-06,0.25,2.25
4,2022-09-08,0.25,2.50
5,2022-11-03,0.25,2.75
6,2023-01-19,0.00,2.75
7,2023-03-09,0.00,2.75
8,2023-05-03,0.25,3.00
9,2023-07-06,0.00,3.00


## USD - RM exchange rate data

In [70]:
df_usd = pd.read_json("usd.json")
df_usd

,date,buying_rate,selling_rate,middle_rate
0,2022-01-03,4.169,4.172,4.1705
1,2022-01-04,4.179,4.183,4.1810
2,2022-01-05,4.190,4.194,4.1920
3,2022-01-06,4.197,4.202,4.1995
4,2022-01-07,4.213,4.216,4.2145
...,...,...,...,...
481,2023-12-21,4.654,4.660,4.6570
482,2023-12-22,4.630,4.640,4.6350
483,2023-12-26,4.613,4.624,4.6185
484,2023-12-27,4.632,4.638,4.6350


# Data preparation

In [6]:
df_sales_by_hour = pd.concat([py_data, cy_data])
df_sales_by_hour["date"] = pd.to_datetime(df_sales_by_hour["date"])
df_sales_by_hour["number_of_transactions"] = pd.to_numeric(df_sales_by_hour["number_of_transactions"])
df_sales_by_hour["sum_check_subtotal"] = pd.to_numeric(df_sales_by_hour["sum_check_subtotal"])
df_sales_by_hour["sum_check_tax"] = pd.to_numeric(df_sales_by_hour["sum_check_tax"])

In [7]:
df_sales_by_hour.dtypes

date                      datetime64[ns]
entry_time_30_mins                object
store_code                        object
casaname                          object
state                             object
trans_type_int                     int64
trans_type                        object
number_of_transactions             int64
sum_check_subtotal               float64
sum_check_tax                    float64
dtype: object

In [8]:
#without transaction type
df_sales_by_hour_grouped_excl_trans_type = df_sales_by_hour.groupby(["date", "store_code", "casaname", "state"])[["number_of_transactions", "sum_check_subtotal", "sum_check_tax"]].agg({"number_of_transactions": ['sum'], "sum_check_subtotal": ['sum'], "sum_check_tax": ['sum']})
df_sales_by_day_restructured_excl_trans_type = df_sales_by_hour_grouped_excl_trans_type.reset_index()

cols = ["date", "store_code", "casaname", "state", "sum_number_of_transactions", "sum_check_subtotal", "sum_check_tax"]
df_sales_by_day_restructured_excl_trans_type.columns = cols

In [9]:
#with transaction type
df_sales_by_hour_grouped_incl_trans_type = df_sales_by_hour.groupby(["date", "store_code", "casaname", "state", "trans_type_int", "trans_type"])[["number_of_transactions", "sum_check_subtotal", "sum_check_tax"]].agg({"number_of_transactions": ['sum'], "sum_check_subtotal": ['sum'], "sum_check_tax": ['sum']})
df_sales_by_day_restructured_incl_trans_type = df_sales_by_hour_grouped_incl_trans_type.reset_index()

cols = ["date", "store_code", "casaname", "state", "trans_type_int", "trans_type", "sum_number_of_transactions", "sum_check_subtotal", "sum_check_tax"]
df_sales_by_day_restructured_incl_trans_type.columns = cols

In [10]:
#upload features file
fact_holidays = pd.read_excel("sales_forecasting_features.xlsx", sheet_name = "fact_holidays")
fact_govt_payday = pd.read_excel("sales_forecasting_features.xlsx", sheet_name = "fact_govt_payday")
fact_priv_payday = pd.read_excel("sales_forecasting_features.xlsx", sheet_name = "fact_priv_payday")
fact_ramadan = pd.read_excel("sales_forecasting_features.xlsx", sheet_name = "fact_ramadan")
fact_school_holidays = pd.read_excel("sales_forecasting_features.xlsx", sheet_name = "fact_school_holidays")
fact_states = pd.read_excel("sales_forecasting_features.xlsx", sheet_name = "fact_states")

In [11]:
#inner join with fact_states
df_sales_by_day_states_excl_trans_type = pd.merge(df_sales_by_day_restructured_excl_trans_type, fact_states, on = 'state', how = 'inner')
df_sales_by_day_states_incl_trans_type = pd.merge(df_sales_by_day_restructured_incl_trans_type, fact_states, on = 'state', how = 'inner')

In [12]:
#left join with fact_holidays
df_sales_by_day_holidays_excl_trans_type = pd.merge(df_sales_by_day_states_excl_trans_type, fact_holidays, left_on = ['date', 'States'], right_on = ["Date", "States"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_holidays_incl_trans_type = pd.merge(df_sales_by_day_states_incl_trans_type, fact_holidays, left_on = ['date', 'States'], right_on = ["Date", "States"], how = 'left', suffixes = (None, '_y'))

In [13]:
#left join with fact_govt_payday
df_sales_by_day_govt_payday_excl_trans_type = pd.merge(df_sales_by_day_holidays_excl_trans_type, fact_govt_payday, left_on = ['date'], right_on = ["pay_date"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_govt_payday_incl_trans_type = pd.merge(df_sales_by_day_holidays_incl_trans_type, fact_govt_payday, left_on = ['date'], right_on = ["pay_date"], how = 'left', suffixes = (None, '_y'))

In [14]:
df_sales_by_day_govt_payday_excl_trans_type = df_sales_by_day_govt_payday_excl_trans_type.drop(["month", "pay_date"], axis = 1)
df_sales_by_day_govt_payday_incl_trans_type = df_sales_by_day_govt_payday_incl_trans_type.drop(["month", "pay_date"], axis = 1)

In [15]:
#inner join with fact_priv_payday
df_sales_by_day_priv_payday_excl_trans_type = pd.merge(df_sales_by_day_govt_payday_excl_trans_type, fact_priv_payday, left_on = ['date'], right_on = ["pay_date"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_priv_payday_incl_trans_type = pd.merge(df_sales_by_day_govt_payday_incl_trans_type, fact_priv_payday, left_on = ['date'], right_on = ["pay_date"], how = 'left', suffixes = (None, '_y'))

In [16]:
df_sales_by_day_priv_payday_excl_trans_type = df_sales_by_day_priv_payday_excl_trans_type.drop(["month", "pay_date"], axis = 1)
df_sales_by_day_priv_payday_incl_trans_type = df_sales_by_day_priv_payday_incl_trans_type.drop(["month", "pay_date"], axis = 1)

In [17]:
#inner join with fact_ramadan
df_sales_by_day_ramadan_excl_trans_type = pd.merge(df_sales_by_day_priv_payday_excl_trans_type, fact_ramadan, left_on = ['date'], right_on = ["date"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_ramadan_incl_trans_type = pd.merge(df_sales_by_day_priv_payday_incl_trans_type, fact_ramadan, left_on = ['date'], right_on = ["date"], how = 'left', suffixes = (None, '_y'))

In [18]:
#inner join with fact_school_holidays
df_sales_by_day_school_holidays_excl_trans_type = pd.merge(df_sales_by_day_ramadan_excl_trans_type, fact_school_holidays, left_on = ['date', 'States'], right_on = ["date", "state"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_school_holidays_incl_trans_type = pd.merge(df_sales_by_day_ramadan_incl_trans_type, fact_school_holidays, left_on = ['date', 'States'], right_on = ["date", "state"], how = 'left', suffixes = (None, '_y'))

In [19]:
df_sales_by_day_school_holidays_excl_trans_type = df_sales_by_day_school_holidays_excl_trans_type.drop(["state_y"], axis = 1)
df_sales_by_day_school_holidays_incl_trans_type = df_sales_by_day_school_holidays_incl_trans_type.drop(["state_y"], axis = 1)

In [20]:
df_sales_by_day_school_holidays_excl_trans_type = df_sales_by_day_school_holidays_excl_trans_type.replace(np.nan, 0)
df_sales_by_day_school_holidays_incl_trans_type = df_sales_by_day_school_holidays_incl_trans_type.replace(np.nan, 0)

In [21]:
df_sales_by_day_school_holidays_excl_trans_type["day"] = df_sales_by_day_school_holidays_excl_trans_type["date"].apply(lambda x: x.strftime('%A'))
df_sales_by_day_school_holidays_incl_trans_type["day"] = df_sales_by_day_school_holidays_incl_trans_type["date"].apply(lambda x: x.strftime('%A'))

In [22]:
#to change position of columns
df_sales_by_day_with_vars_excl_trans_type = df_sales_by_day_school_holidays_excl_trans_type[["date", "day", "store_code", "casaname", "state", "sum_number_of_transactions", "sum_check_subtotal", "sum_check_tax", "States", "Holiday", "govt_pay_date_remark", 	"priv_pay_date_remark", "ramadan_week", "ramadan_remark", "school_holidays_remark"]]
df_sales_by_day_with_vars_incl_trans_type = df_sales_by_day_school_holidays_incl_trans_type[["date", "day", "store_code", "casaname", "state", "trans_type_int", "trans_type", "sum_number_of_transactions", "sum_check_subtotal", "sum_check_tax", "States", "Holiday", "govt_pay_date_remark", 	"priv_pay_date_remark", "ramadan_week", "ramadan_remark", "school_holidays_remark"]]

In [23]:
df_sales_by_day_with_vars_excl_trans_type = df_sales_by_day_with_vars_excl_trans_type.copy()
df_sales_by_day_with_vars_excl_trans_type["weekend"] = df_sales_by_day_with_vars_excl_trans_type['day'].apply(lambda x: 1 if x == "Saturday" or x == "Sunday" else 0)
df_sales_by_day_with_vars_excl_trans_type = df_sales_by_day_with_vars_excl_trans_type[["date", "day", "weekend", "store_code", "casaname", "state", "sum_number_of_transactions", "sum_check_subtotal", "sum_check_tax", "States", "Holiday", "govt_pay_date_remark", "priv_pay_date_remark", "ramadan_week", "ramadan_remark", "school_holidays_remark"]]

df_sales_by_day_with_vars_incl_trans_type = df_sales_by_day_with_vars_incl_trans_type.copy()
df_sales_by_day_with_vars_incl_trans_type["weekend"] = df_sales_by_day_with_vars_incl_trans_type['day'].apply(lambda x: 1 if x == "Saturday" or x == "Sunday" else 0)
df_sales_by_day_with_vars_incl_trans_type = df_sales_by_day_with_vars_incl_trans_type[["date", "day", "weekend", "store_code", "casaname", "state", "trans_type_int", "trans_type", "sum_number_of_transactions", "sum_check_subtotal", "sum_check_tax", "States", "Holiday", "govt_pay_date_remark", "priv_pay_date_remark", "ramadan_week", "ramadan_remark", "school_holidays_remark"]]

In [24]:
#add week number of the month
df_sales_by_day_with_vars_excl_trans_type = df_sales_by_day_with_vars_excl_trans_type.copy()
df_sales_by_day_with_vars_excl_trans_type['week_of_mon'] = (df_sales_by_day_with_vars_excl_trans_type['date'].dt.day - 1) // 7 + 1

df_sales_by_day_with_vars_incl_trans_type = df_sales_by_day_with_vars_incl_trans_type.copy()
df_sales_by_day_with_vars_incl_trans_type['week_of_mon'] = (df_sales_by_day_with_vars_incl_trans_type['date'].dt.day - 1) // 7 + 1

In [25]:
#add number of the month
df_sales_by_day_with_vars_excl_trans_type['num_of_mon'] = df_sales_by_day_with_vars_excl_trans_type["date"].dt.month
df_sales_by_day_with_vars_incl_trans_type['num_of_mon'] = df_sales_by_day_with_vars_incl_trans_type["date"].dt.month

In [26]:
df_sales_by_day_with_vars_excl_trans_type = df_sales_by_day_with_vars_excl_trans_type[["date", "day", "weekend", "week_of_mon", "num_of_mon", "store_code", "casaname", "state", "sum_number_of_transactions", "sum_check_subtotal", "sum_check_tax", "States", "Holiday", "govt_pay_date_remark", "priv_pay_date_remark", "ramadan_week", "ramadan_remark", "school_holidays_remark"]]
df_sales_by_day_with_vars_incl_trans_type = df_sales_by_day_with_vars_incl_trans_type[["date", "day", "weekend", "week_of_mon", "num_of_mon", "store_code", "casaname", "state", "trans_type_int", "trans_type", "sum_number_of_transactions", "sum_check_subtotal", "sum_check_tax", "States", "Holiday", "govt_pay_date_remark", "priv_pay_date_remark", "ramadan_week", "ramadan_remark", "school_holidays_remark"]]

In [27]:
#this is a function to identify long weekends

df_sales_by_day_with_vars_excl_trans_type = df_sales_by_day_with_vars_excl_trans_type.copy()
long = (df_sales_by_day_with_vars_excl_trans_type['Holiday'] != 0) | ((df_sales_by_day_with_vars_excl_trans_type['day'] == 'Saturday') | (df_sales_by_day_with_vars_excl_trans_type['day'] == 'Sunday'))
s = long.ne(long.shift()).cumsum()
df_sales_by_day_with_vars_excl_trans_type['long_weekend'] = np.where((s.map(s.value_counts()) > 2) & long, 1, 0)

df_sales_by_day_with_vars_incl_trans_type = df_sales_by_day_with_vars_incl_trans_type.copy()
long = (df_sales_by_day_with_vars_incl_trans_type['Holiday'] != 0) | ((df_sales_by_day_with_vars_incl_trans_type['day'] == 'Saturday') | (df_sales_by_day_with_vars_incl_trans_type['day'] == 'Sunday'))
s = long.ne(long.shift()).cumsum()
df_sales_by_day_with_vars_incl_trans_type['long_weekend'] = np.where((s.map(s.value_counts()) > 2) & long, 1, 0)

In [28]:
df_sales_by_day_with_vars_excl_trans_type = df_sales_by_day_with_vars_excl_trans_type.drop_duplicates().reset_index().drop(["index"], axis = 1)
df_sales_by_day_with_vars_incl_trans_type = df_sales_by_day_with_vars_incl_trans_type.drop_duplicates().reset_index().drop(["index"], axis = 1)

In [29]:
#1 Utama: all, and dine in/take-away
df_1utama_excl_trans_type = df_sales_by_day_with_vars_excl_trans_type[(df_sales_by_day_with_vars_excl_trans_type["casaname"] == "1 UTAMA")]
df_1utama_incl_trans_type_dinein = df_sales_by_day_with_vars_incl_trans_type[(df_sales_by_day_with_vars_incl_trans_type["casaname"] == "1 UTAMA") & (df_sales_by_day_with_vars_incl_trans_type["trans_type"] == "01 - Dine-In")]
df_1utama_incl_trans_type_takeaway = df_sales_by_day_with_vars_incl_trans_type[(df_sales_by_day_with_vars_incl_trans_type["casaname"] == "1 UTAMA") & (df_sales_by_day_with_vars_incl_trans_type["trans_type"] == "02 - Take Away")]

#AEON AU2: all, and dine in/take-away
df_aeonau2_excl_trans_type = df_sales_by_day_with_vars_excl_trans_type[(df_sales_by_day_with_vars_excl_trans_type["casaname"] == "AEON AU2")]
df_aeonau2_incl_trans_type_dinein = df_sales_by_day_with_vars_incl_trans_type[(df_sales_by_day_with_vars_incl_trans_type["casaname"] == "AEON AU2") & (df_sales_by_day_with_vars_incl_trans_type["trans_type"] == "01 - Dine-In")]
df_aeonau2_incl_trans_type_takeaway = df_sales_by_day_with_vars_incl_trans_type[(df_sales_by_day_with_vars_incl_trans_type["casaname"] == "AEON AU2") & (df_sales_by_day_with_vars_incl_trans_type["trans_type"] == "02 - Take Away")]

#AEON BANDARAYA MELAKA: all, and dine in/take-away
df_aeonbandarayamelaka_excl_trans_type = df_sales_by_day_with_vars_excl_trans_type[(df_sales_by_day_with_vars_excl_trans_type["casaname"] == "AEON BANDARAYA MELAKA")]
df_aeonbandarayamelaka_incl_trans_type_dinein = df_sales_by_day_with_vars_incl_trans_type[(df_sales_by_day_with_vars_incl_trans_type["casaname"] == "AEON BANDARAYA MELAKA") & (df_sales_by_day_with_vars_incl_trans_type["trans_type"] == "01 - Dine-In")]
df_aeonbandarayamelaka_incl_trans_type_takeaway = df_sales_by_day_with_vars_incl_trans_type[(df_sales_by_day_with_vars_incl_trans_type["casaname"] == "AEON BANDARAYA MELAKA") & (df_sales_by_day_with_vars_incl_trans_type["trans_type"] == "02 - Take Away")]

#AEON BUKIT MERTAJAM: all, and dine in/take-away
df_aeonbukitmertajam_excl_trans_type = df_sales_by_day_with_vars_excl_trans_type[(df_sales_by_day_with_vars_excl_trans_type["casaname"] == "AEON BUKIT MERTAJAM")]
df_aeonbukitmertajam_incl_trans_type_dinein = df_sales_by_day_with_vars_incl_trans_type[(df_sales_by_day_with_vars_incl_trans_type["casaname"] == "AEON BUKIT MERTAJAM") & (df_sales_by_day_with_vars_incl_trans_type["trans_type"] == "01 - Dine-In")]
df_aeonbukitmertajam_incl_trans_type_takeaway = df_sales_by_day_with_vars_incl_trans_type[(df_sales_by_day_with_vars_incl_trans_type["casaname"] == "AEON BUKIT MERTAJAM") & (df_sales_by_day_with_vars_incl_trans_type["trans_type"] == "02 - Take Away")]

#AEON BUKIT TINGGI: all, and dine in/take-away
df_aeonbukittinggi_excl_trans_type = df_sales_by_day_with_vars_excl_trans_type[(df_sales_by_day_with_vars_excl_trans_type["casaname"] == "AEON BUKIT TINGGI")]
df_aeonbukittinggi_incl_trans_type_dinein = df_sales_by_day_with_vars_incl_trans_type[(df_sales_by_day_with_vars_incl_trans_type["casaname"] == "AEON BUKIT TINGGI") & (df_sales_by_day_with_vars_incl_trans_type["trans_type"] == "01 - Dine-In")]
df_aeonbukittinggi_incl_trans_type_takeaway = df_sales_by_day_with_vars_incl_trans_type[(df_sales_by_day_with_vars_incl_trans_type["casaname"] == "AEON BUKIT TINGGI") & (df_sales_by_day_with_vars_incl_trans_type["trans_type"] == "02 - Take Away")]

In [30]:
#generate list of public holidays for selected branches
holidays_1utama = list(set(list(df_1utama_excl_trans_type["Holiday"])))
holidays_1utama = holidays_1utama[1:]

holidays_aeonau2 = list(set(list(df_aeonau2_excl_trans_type["Holiday"])))
holidays_aeonau2 = holidays_aeonau2[1:]

holidays_aeonbandarayamelaka = list(set(list(df_aeonbandarayamelaka_excl_trans_type["Holiday"])))
holidays_aeonbandarayamelaka = holidays_aeonbandarayamelaka[1:]

holidays_aeonbukitmertajam = list(set(list(df_aeonbukitmertajam_excl_trans_type["Holiday"])))
holidays_aeonbukitmertajam = holidays_aeonbukitmertajam[1:]

holidays_aeonbukittinggi = list(set(list(df_aeonbukittinggi_excl_trans_type["Holiday"])))
holidays_aeonbukittinggi = holidays_aeonbukittinggi[1:]

In [31]:
#1 UTAMA: find and add holiday weights
def generate_public_holiday_weights(holiday_name):
    df_diamond = df_1utama_excl_trans_type[(df_1utama_excl_trans_type["Holiday"] == str(holiday_name)) & (df_1utama_excl_trans_type["date"] <= '2023-06-06')][["day", "weekend", "store_code", "casaname", "state", "sum_check_subtotal", "school_holidays_remark"]]
    df_gem = df_1utama_excl_trans_type[
    (df_1utama_excl_trans_type["day"] == df_diamond["day"].values[0]) &
    (df_1utama_excl_trans_type["weekend"] == df_diamond["weekend"].values[0]) &
    (df_1utama_excl_trans_type["store_code"] == df_diamond["store_code"].values[0]) &
    (df_1utama_excl_trans_type["casaname"] == df_diamond["casaname"].values[0]) &
    (df_1utama_excl_trans_type["state"] == df_diamond["state"].values[0]) &
    (df_1utama_excl_trans_type["Holiday"] == 0) &
    (df_1utama_excl_trans_type["date"] <= '2023-06-06')
    ]
    try:
      weight = round((df_diamond['sum_check_subtotal'].values[0] - df_gem['sum_check_subtotal'].values[0])/(df_gem['sum_check_subtotal'].values[0]), 4)
    except:
      weight = 0
    return(weight)

df_holiday_weights_1utama = pd.DataFrame(holidays_1utama, columns = ["holidays"])
df_holiday_weights_1utama["weights"] = df_holiday_weights_1utama["holidays"].apply(lambda x: generate_public_holiday_weights(x))

df_sales_by_day_1utama_excl_trans_type = pd.merge(df_1utama_excl_trans_type, df_holiday_weights_1utama, left_on = ['Holiday'], right_on = ["holidays"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_1utama_excl_trans_type = df_sales_by_day_1utama_excl_trans_type.drop(["holidays"], axis = 1)
df_sales_by_day_1utama_excl_trans_type = df_sales_by_day_1utama_excl_trans_type.replace(np.nan, 0)


def generate_public_holiday_weights(holiday_name):
    df_diamond = df_1utama_incl_trans_type_dinein[(df_1utama_incl_trans_type_dinein["Holiday"] == str(holiday_name)) & (df_1utama_incl_trans_type_dinein["date"] <= '2023-06-06')][["day", "weekend", "store_code", "casaname", "state", "sum_check_subtotal", "school_holidays_remark"]]
    df_gem = df_1utama_incl_trans_type_dinein[
    (df_1utama_incl_trans_type_dinein["day"] == df_diamond["day"].values[0]) &
    (df_1utama_incl_trans_type_dinein["weekend"] == df_diamond["weekend"].values[0]) &
    (df_1utama_incl_trans_type_dinein["store_code"] == df_diamond["store_code"].values[0]) &
    (df_1utama_incl_trans_type_dinein["casaname"] == df_diamond["casaname"].values[0]) &
    (df_1utama_incl_trans_type_dinein["state"] == df_diamond["state"].values[0]) &
    (df_1utama_incl_trans_type_dinein["Holiday"] == 0) &
    (df_1utama_incl_trans_type_dinein["date"] <= '2023-06-06')
    ]
    try:
      weight = round((df_diamond['sum_check_subtotal'].values[0] - df_gem['sum_check_subtotal'].values[0])/(df_gem['sum_check_subtotal'].values[0]), 4)
    except:
      weight = 0
    return(weight)

df_holiday_weights_1utama = pd.DataFrame(holidays_1utama, columns = ["holidays"])
df_holiday_weights_1utama["weights"] = df_holiday_weights_1utama["holidays"].apply(lambda x: generate_public_holiday_weights(x))

df_sales_by_day_1utama_incl_trans_type_dinein = pd.merge(df_1utama_incl_trans_type_dinein, df_holiday_weights_1utama, left_on = ['Holiday'], right_on = ["holidays"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_1utama_incl_trans_type_dinein = df_sales_by_day_1utama_incl_trans_type_dinein.drop(["holidays"], axis = 1)
df_sales_by_day_1utama_incl_trans_type_dinein = df_sales_by_day_1utama_incl_trans_type_dinein.replace(np.nan, 0)


def generate_public_holiday_weights(holiday_name):
    df_diamond = df_1utama_incl_trans_type_takeaway[(df_1utama_incl_trans_type_takeaway["Holiday"] == str(holiday_name)) & (df_1utama_incl_trans_type_takeaway["date"] <= '2023-06-06')][["day", "weekend", "store_code", "casaname", "state", "sum_check_subtotal", "school_holidays_remark"]]
    df_gem = df_1utama_incl_trans_type_takeaway[
    (df_1utama_incl_trans_type_takeaway["day"] == df_diamond["day"].values[0]) &
    (df_1utama_incl_trans_type_takeaway["weekend"] == df_diamond["weekend"].values[0]) &
    (df_1utama_incl_trans_type_takeaway["store_code"] == df_diamond["store_code"].values[0]) &
    (df_1utama_incl_trans_type_takeaway["casaname"] == df_diamond["casaname"].values[0]) &
    (df_1utama_incl_trans_type_takeaway["state"] == df_diamond["state"].values[0]) &
    (df_1utama_incl_trans_type_takeaway["Holiday"] == 0) &
    (df_1utama_incl_trans_type_takeaway["date"] <= '2023-06-06')
    ]
    try:
      weight = round((df_diamond['sum_check_subtotal'].values[0] - df_gem['sum_check_subtotal'].values[0])/(df_gem['sum_check_subtotal'].values[0]), 4)
    except:
      weight = 0
    return(weight)

df_holiday_weights_1utama = pd.DataFrame(holidays_1utama, columns = ["holidays"])
df_holiday_weights_1utama["weights"] = df_holiday_weights_1utama["holidays"].apply(lambda x: generate_public_holiday_weights(x))

df_sales_by_day_1utama_incl_trans_type_takeaway = pd.merge(df_1utama_incl_trans_type_takeaway, df_holiday_weights_1utama, left_on = ['Holiday'], right_on = ["holidays"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_1utama_incl_trans_type_takeaway = df_sales_by_day_1utama_incl_trans_type_takeaway.drop(["holidays"], axis = 1)
df_sales_by_day_1utama_incl_trans_type_takeaway = df_sales_by_day_1utama_incl_trans_type_takeaway.replace(np.nan, 0)

In [32]:
#AEON AU2: find and add holiday weights
def generate_public_holiday_weights(holiday_name):
    df_diamond = df_aeonau2_excl_trans_type[(df_aeonau2_excl_trans_type["Holiday"] == str(holiday_name)) & (df_aeonau2_excl_trans_type["date"] <= '2023-06-06')][["day", "weekend", "store_code", "casaname", "state", "sum_check_subtotal", "school_holidays_remark"]]
    df_gem = df_aeonau2_excl_trans_type[
    (df_aeonau2_excl_trans_type["day"] == df_diamond["day"].values[0]) &
    (df_aeonau2_excl_trans_type["weekend"] == df_diamond["weekend"].values[0]) &
    (df_aeonau2_excl_trans_type["store_code"] == df_diamond["store_code"].values[0]) &
    (df_aeonau2_excl_trans_type["casaname"] == df_diamond["casaname"].values[0]) &
    (df_aeonau2_excl_trans_type["state"] == df_diamond["state"].values[0]) &
    (df_aeonau2_excl_trans_type["Holiday"] == 0) &
    (df_aeonau2_excl_trans_type["date"] <= '2023-06-06')
    ]
    try:
      weight = round((df_diamond['sum_check_subtotal'].values[0] - df_gem['sum_check_subtotal'].values[0])/(df_gem['sum_check_subtotal'].values[0]), 4)
    except:
      weight = 0
    return(weight)

df_holiday_weights_aeonau2 = pd.DataFrame(holidays_aeonau2, columns = ["holidays"])
df_holiday_weights_aeonau2["weights"] = df_holiday_weights_aeonau2["holidays"].apply(lambda x: generate_public_holiday_weights(x))

df_sales_by_day_aeonau2_excl_trans_type = pd.merge(df_aeonau2_excl_trans_type, df_holiday_weights_aeonau2, left_on = ['Holiday'], right_on = ["holidays"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_aeonau2_excl_trans_type = df_sales_by_day_aeonau2_excl_trans_type.drop(["holidays"], axis = 1)
df_sales_by_day_aeonau2_excl_trans_type = df_sales_by_day_aeonau2_excl_trans_type.replace(np.nan, 0)


def generate_public_holiday_weights(holiday_name):
    df_diamond = df_aeonau2_incl_trans_type_dinein[(df_aeonau2_incl_trans_type_dinein["Holiday"] == str(holiday_name)) & (df_aeonau2_incl_trans_type_dinein["date"] <= '2023-06-06')][["day", "weekend", "store_code", "casaname", "state", "sum_check_subtotal", "school_holidays_remark"]]
    df_gem = df_aeonau2_incl_trans_type_dinein[
    (df_aeonau2_incl_trans_type_dinein["day"] == df_diamond["day"].values[0]) &
    (df_aeonau2_incl_trans_type_dinein["weekend"] == df_diamond["weekend"].values[0]) &
    (df_aeonau2_incl_trans_type_dinein["store_code"] == df_diamond["store_code"].values[0]) &
    (df_aeonau2_incl_trans_type_dinein["casaname"] == df_diamond["casaname"].values[0]) &
    (df_aeonau2_incl_trans_type_dinein["state"] == df_diamond["state"].values[0]) &
    (df_aeonau2_incl_trans_type_dinein["Holiday"] == 0) &
    (df_aeonau2_incl_trans_type_dinein["date"] <= '2023-06-06')
    ]
    try:
      weight = round((df_diamond['sum_check_subtotal'].values[0] - df_gem['sum_check_subtotal'].values[0])/(df_gem['sum_check_subtotal'].values[0]), 4)
    except:
      weight = 0
    return(weight)

df_holiday_weights_aeonau2 = pd.DataFrame(holidays_aeonau2, columns = ["holidays"])
df_holiday_weights_aeonau2["weights"] = df_holiday_weights_aeonau2["holidays"].apply(lambda x: generate_public_holiday_weights(x))

df_sales_by_day_aeonau2_incl_trans_type_dinein = pd.merge(df_aeonau2_incl_trans_type_dinein, df_holiday_weights_aeonau2, left_on = ['Holiday'], right_on = ["holidays"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_aeonau2_incl_trans_type_dinein = df_sales_by_day_aeonau2_incl_trans_type_dinein.drop(["holidays"], axis = 1)
df_sales_by_day_aeonau2_incl_trans_type_dinein = df_sales_by_day_aeonau2_incl_trans_type_dinein.replace(np.nan, 0)


def generate_public_holiday_weights(holiday_name):
    df_diamond = df_aeonau2_incl_trans_type_takeaway[(df_aeonau2_incl_trans_type_takeaway["Holiday"] == str(holiday_name)) & (df_aeonau2_incl_trans_type_takeaway["date"] <= '2023-06-06')][["day", "weekend", "store_code", "casaname", "state", "sum_check_subtotal", "school_holidays_remark"]]
    df_gem = df_aeonau2_incl_trans_type_takeaway[
    (df_aeonau2_incl_trans_type_takeaway["day"] == df_diamond["day"].values[0]) &
    (df_aeonau2_incl_trans_type_takeaway["weekend"] == df_diamond["weekend"].values[0]) &
    (df_aeonau2_incl_trans_type_takeaway["store_code"] == df_diamond["store_code"].values[0]) &
    (df_aeonau2_incl_trans_type_takeaway["casaname"] == df_diamond["casaname"].values[0]) &
    (df_aeonau2_incl_trans_type_takeaway["state"] == df_diamond["state"].values[0]) &
    (df_aeonau2_incl_trans_type_takeaway["Holiday"] == 0) &
    (df_aeonau2_incl_trans_type_takeaway["date"] <= '2023-06-06')
    ]
    try:
      weight = round((df_diamond['sum_check_subtotal'].values[0] - df_gem['sum_check_subtotal'].values[0])/(df_gem['sum_check_subtotal'].values[0]), 4)
    except:
      weight = 0
    return(weight)

df_holiday_weights_aeonau2 = pd.DataFrame(holidays_aeonau2, columns = ["holidays"])
df_holiday_weights_aeonau2["weights"] = df_holiday_weights_aeonau2["holidays"].apply(lambda x: generate_public_holiday_weights(x))

df_sales_by_day_aeonau2_incl_trans_type_takeaway = pd.merge(df_aeonau2_incl_trans_type_takeaway, df_holiday_weights_aeonau2, left_on = ['Holiday'], right_on = ["holidays"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_aeonau2_incl_trans_type_takeaway = df_sales_by_day_aeonau2_incl_trans_type_takeaway.drop(["holidays"], axis = 1)
df_sales_by_day_aeonau2_incl_trans_type_takeaway = df_sales_by_day_aeonau2_incl_trans_type_takeaway.replace(np.nan, 0)

In [33]:
#AEON BANDARAYA MELAKA: find and add holiday weights
def generate_public_holiday_weights(holiday_name):
    df_diamond = df_aeonbandarayamelaka_excl_trans_type[(df_aeonbandarayamelaka_excl_trans_type["Holiday"] == str(holiday_name)) & (df_aeonbandarayamelaka_excl_trans_type["date"] <= '2023-06-06')][["day", "weekend", "store_code", "casaname", "state", "sum_check_subtotal", "school_holidays_remark"]]
    df_gem = df_aeonbandarayamelaka_excl_trans_type[
    (df_aeonbandarayamelaka_excl_trans_type["day"] == df_diamond["day"].values[0]) &
    (df_aeonbandarayamelaka_excl_trans_type["weekend"] == df_diamond["weekend"].values[0]) &
    (df_aeonbandarayamelaka_excl_trans_type["store_code"] == df_diamond["store_code"].values[0]) &
    (df_aeonbandarayamelaka_excl_trans_type["casaname"] == df_diamond["casaname"].values[0]) &
    (df_aeonbandarayamelaka_excl_trans_type["state"] == df_diamond["state"].values[0]) &
    (df_aeonbandarayamelaka_excl_trans_type["Holiday"] == 0) &
    (df_aeonbandarayamelaka_excl_trans_type["date"] <= '2023-06-06')
    ]
    try:
      weight = round((df_diamond['sum_check_subtotal'].values[0] - df_gem['sum_check_subtotal'].values[0])/(df_gem['sum_check_subtotal'].values[0]), 4)
    except:
      weight = 0
    return(weight)

df_holiday_weights_aeonbandarayamelaka = pd.DataFrame(holidays_aeonbandarayamelaka, columns = ["holidays"])
df_holiday_weights_aeonbandarayamelaka["weights"] = df_holiday_weights_aeonbandarayamelaka["holidays"].apply(lambda x: generate_public_holiday_weights(x))

df_sales_by_day_aeonbandarayamelaka_excl_trans_type = pd.merge(df_aeonbandarayamelaka_excl_trans_type, df_holiday_weights_aeonbandarayamelaka, left_on = ['Holiday'], right_on = ["holidays"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_aeonbandarayamelaka_excl_trans_type = df_sales_by_day_aeonbandarayamelaka_excl_trans_type.drop(["holidays"], axis = 1)
df_sales_by_day_aeonbandarayamelaka_excl_trans_type = df_sales_by_day_aeonbandarayamelaka_excl_trans_type.replace(np.nan, 0)


def generate_public_holiday_weights(holiday_name):
    df_diamond = df_aeonbandarayamelaka_incl_trans_type_dinein[(df_aeonbandarayamelaka_incl_trans_type_dinein["Holiday"] == str(holiday_name)) & (df_aeonbandarayamelaka_incl_trans_type_dinein["date"] <= '2023-06-06')][["day", "weekend", "store_code", "casaname", "state", "sum_check_subtotal", "school_holidays_remark"]]
    df_gem = df_aeonbandarayamelaka_incl_trans_type_dinein[
    (df_aeonbandarayamelaka_incl_trans_type_dinein["day"] == df_diamond["day"].values[0]) &
    (df_aeonbandarayamelaka_incl_trans_type_dinein["weekend"] == df_diamond["weekend"].values[0]) &
    (df_aeonbandarayamelaka_incl_trans_type_dinein["store_code"] == df_diamond["store_code"].values[0]) &
    (df_aeonbandarayamelaka_incl_trans_type_dinein["casaname"] == df_diamond["casaname"].values[0]) &
    (df_aeonbandarayamelaka_incl_trans_type_dinein["state"] == df_diamond["state"].values[0]) &
    (df_aeonbandarayamelaka_incl_trans_type_dinein["Holiday"] == 0) &
    (df_aeonbandarayamelaka_incl_trans_type_dinein["date"] <= '2023-06-06')
    ]
    try:
      weight = round((df_diamond['sum_check_subtotal'].values[0] - df_gem['sum_check_subtotal'].values[0])/(df_gem['sum_check_subtotal'].values[0]), 4)
    except:
      weight = 0
    return(weight)

df_holiday_weights_aeonbandarayamelaka = pd.DataFrame(holidays_aeonbandarayamelaka, columns = ["holidays"])
df_holiday_weights_aeonbandarayamelaka["weights"] = df_holiday_weights_aeonbandarayamelaka["holidays"].apply(lambda x: generate_public_holiday_weights(x))

df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein = pd.merge(df_aeonbandarayamelaka_incl_trans_type_dinein, df_holiday_weights_aeonbandarayamelaka, left_on = ['Holiday'], right_on = ["holidays"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein.drop(["holidays"], axis = 1)
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein.replace(np.nan, 0)


def generate_public_holiday_weights(holiday_name):
    df_diamond = df_aeonbandarayamelaka_incl_trans_type_takeaway[(df_aeonbandarayamelaka_incl_trans_type_takeaway["Holiday"] == str(holiday_name)) & (df_aeonbandarayamelaka_incl_trans_type_takeaway["date"] <= '2023-06-06')][["day", "weekend", "store_code", "casaname", "state", "sum_check_subtotal", "school_holidays_remark"]]
    df_gem = df_aeonbandarayamelaka_incl_trans_type_takeaway[
    (df_aeonbandarayamelaka_incl_trans_type_takeaway["day"] == df_diamond["day"].values[0]) &
    (df_aeonbandarayamelaka_incl_trans_type_takeaway["weekend"] == df_diamond["weekend"].values[0]) &
    (df_aeonbandarayamelaka_incl_trans_type_takeaway["store_code"] == df_diamond["store_code"].values[0]) &
    (df_aeonbandarayamelaka_incl_trans_type_takeaway["casaname"] == df_diamond["casaname"].values[0]) &
    (df_aeonbandarayamelaka_incl_trans_type_takeaway["state"] == df_diamond["state"].values[0]) &
    (df_aeonbandarayamelaka_incl_trans_type_takeaway["Holiday"] == 0) &
    (df_aeonbandarayamelaka_incl_trans_type_takeaway["date"] <= '2023-06-06')
    ]
    try:
      weight = round((df_diamond['sum_check_subtotal'].values[0] - df_gem['sum_check_subtotal'].values[0])/(df_gem['sum_check_subtotal'].values[0]), 4)
    except:
      weight = 0
    return(weight)

df_holiday_weights_aeonbandarayamelaka = pd.DataFrame(holidays_aeonbandarayamelaka, columns = ["holidays"])
df_holiday_weights_aeonbandarayamelaka["weights"] = df_holiday_weights_aeonbandarayamelaka["holidays"].apply(lambda x: generate_public_holiday_weights(x))

df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway = pd.merge(df_aeonbandarayamelaka_incl_trans_type_takeaway, df_holiday_weights_aeonbandarayamelaka, left_on = ['Holiday'], right_on = ["holidays"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway.drop(["holidays"], axis = 1)
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway.replace(np.nan, 0)

In [34]:
#AEON BUKIT MERTAJAM: find and add holiday weights
def generate_public_holiday_weights(holiday_name):
    df_diamond = df_aeonbukitmertajam_excl_trans_type[(df_aeonbukitmertajam_excl_trans_type["Holiday"] == str(holiday_name)) & (df_aeonbukitmertajam_excl_trans_type["date"] <= '2023-06-06')][["day", "weekend", "store_code", "casaname", "state", "sum_check_subtotal", "school_holidays_remark"]]
    df_gem = df_aeonbukitmertajam_excl_trans_type[
    (df_aeonbukitmertajam_excl_trans_type["day"] == df_diamond["day"].values[0]) &
    (df_aeonbukitmertajam_excl_trans_type["weekend"] == df_diamond["weekend"].values[0]) &
    (df_aeonbukitmertajam_excl_trans_type["store_code"] == df_diamond["store_code"].values[0]) &
    (df_aeonbukitmertajam_excl_trans_type["casaname"] == df_diamond["casaname"].values[0]) &
    (df_aeonbukitmertajam_excl_trans_type["state"] == df_diamond["state"].values[0]) &
    (df_aeonbukitmertajam_excl_trans_type["Holiday"] == 0) &
    (df_aeonbukitmertajam_excl_trans_type["date"] <= '2023-06-06')
    ]
    try:
      weight = round((df_diamond['sum_check_subtotal'].values[0] - df_gem['sum_check_subtotal'].values[0])/(df_gem['sum_check_subtotal'].values[0]), 4)
    except:
      weight = 0
    return(weight)

df_holiday_weights_aeonbukitmertajam = pd.DataFrame(holidays_aeonbukitmertajam, columns = ["holidays"])
df_holiday_weights_aeonbukitmertajam["weights"] = df_holiday_weights_aeonbukitmertajam["holidays"].apply(lambda x: generate_public_holiday_weights(x))

df_sales_by_day_aeonbukitmertajam_excl_trans_type = pd.merge(df_aeonbukitmertajam_excl_trans_type, df_holiday_weights_aeonbukitmertajam, left_on = ['Holiday'], right_on = ["holidays"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_aeonbukitmertajam_excl_trans_type = df_sales_by_day_aeonbukitmertajam_excl_trans_type.drop(["holidays"], axis = 1)
df_sales_by_day_aeonbukitmertajam_excl_trans_type = df_sales_by_day_aeonbukitmertajam_excl_trans_type.replace(np.nan, 0)


def generate_public_holiday_weights(holiday_name):
    df_diamond = df_aeonbukitmertajam_incl_trans_type_dinein[(df_aeonbukitmertajam_incl_trans_type_dinein["Holiday"] == str(holiday_name)) & (df_aeonbukitmertajam_incl_trans_type_dinein["date"] <= '2023-06-06')][["day", "weekend", "store_code", "casaname", "state", "sum_check_subtotal", "school_holidays_remark"]]
    df_gem = df_aeonbukitmertajam_incl_trans_type_dinein[
    (df_aeonbukitmertajam_incl_trans_type_dinein["day"] == df_diamond["day"].values[0]) &
    (df_aeonbukitmertajam_incl_trans_type_dinein["weekend"] == df_diamond["weekend"].values[0]) &
    (df_aeonbukitmertajam_incl_trans_type_dinein["store_code"] == df_diamond["store_code"].values[0]) &
    (df_aeonbukitmertajam_incl_trans_type_dinein["casaname"] == df_diamond["casaname"].values[0]) &
    (df_aeonbukitmertajam_incl_trans_type_dinein["state"] == df_diamond["state"].values[0]) &
    (df_aeonbukitmertajam_incl_trans_type_dinein["Holiday"] == 0) &
    (df_aeonbukitmertajam_incl_trans_type_dinein["date"] <= '2023-06-06')
    ]
    try:
      weight = round((df_diamond['sum_check_subtotal'].values[0] - df_gem['sum_check_subtotal'].values[0])/(df_gem['sum_check_subtotal'].values[0]), 4)
    except:
      weight = 0
    return(weight)

df_holiday_weights_aeonbukitmertajam = pd.DataFrame(holidays_aeonbukitmertajam, columns = ["holidays"])
df_holiday_weights_aeonbukitmertajam["weights"] = df_holiday_weights_aeonbukitmertajam["holidays"].apply(lambda x: generate_public_holiday_weights(x))

df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein = pd.merge(df_aeonbukitmertajam_incl_trans_type_dinein, df_holiday_weights_aeonbukitmertajam, left_on = ['Holiday'], right_on = ["holidays"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein = df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein.drop(["holidays"], axis = 1)
df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein = df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein.replace(np.nan, 0)


def generate_public_holiday_weights(holiday_name):
    df_diamond = df_aeonbukitmertajam_incl_trans_type_takeaway[(df_aeonbukitmertajam_incl_trans_type_takeaway["Holiday"] == str(holiday_name)) & (df_aeonbukitmertajam_incl_trans_type_takeaway["date"] <= '2023-06-06')][["day", "weekend", "store_code", "casaname", "state", "sum_check_subtotal", "school_holidays_remark"]]
    df_gem = df_aeonbukitmertajam_incl_trans_type_takeaway[
    (df_aeonbukitmertajam_incl_trans_type_takeaway["day"] == df_diamond["day"].values[0]) &
    (df_aeonbukitmertajam_incl_trans_type_takeaway["weekend"] == df_diamond["weekend"].values[0]) &
    (df_aeonbukitmertajam_incl_trans_type_takeaway["store_code"] == df_diamond["store_code"].values[0]) &
    (df_aeonbukitmertajam_incl_trans_type_takeaway["casaname"] == df_diamond["casaname"].values[0]) &
    (df_aeonbukitmertajam_incl_trans_type_takeaway["state"] == df_diamond["state"].values[0]) &
    (df_aeonbukitmertajam_incl_trans_type_takeaway["Holiday"] == 0) &
    (df_aeonbukitmertajam_incl_trans_type_takeaway["date"] <= '2023-06-06')
    ]
    try:
      weight = round((df_diamond['sum_check_subtotal'].values[0] - df_gem['sum_check_subtotal'].values[0])/(df_gem['sum_check_subtotal'].values[0]), 4)
    except:
      weight = 0
    return(weight)

df_holiday_weights_aeonbukitmertajam = pd.DataFrame(holidays_aeonbukitmertajam, columns = ["holidays"])
df_holiday_weights_aeonbukitmertajam["weights"] = df_holiday_weights_aeonbukitmertajam["holidays"].apply(lambda x: generate_public_holiday_weights(x))

df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway = pd.merge(df_aeonbukitmertajam_incl_trans_type_takeaway, df_holiday_weights_aeonbukitmertajam, left_on = ['Holiday'], right_on = ["holidays"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway = df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway.drop(["holidays"], axis = 1)
df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway = df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway.replace(np.nan, 0)

In [35]:
#AEON BUKIT TINGGI: find and add holiday weights
def generate_public_holiday_weights(holiday_name):
    df_diamond = df_aeonbukittinggi_excl_trans_type[(df_aeonbukittinggi_excl_trans_type["Holiday"] == str(holiday_name)) & (df_aeonbukittinggi_excl_trans_type["date"] <= '2023-06-06')][["day", "weekend", "store_code", "casaname", "state", "sum_check_subtotal", "school_holidays_remark"]]
    df_gem = df_aeonbukittinggi_excl_trans_type[
    (df_aeonbukittinggi_excl_trans_type["day"] == df_diamond["day"].values[0]) &
    (df_aeonbukittinggi_excl_trans_type["weekend"] == df_diamond["weekend"].values[0]) &
    (df_aeonbukittinggi_excl_trans_type["store_code"] == df_diamond["store_code"].values[0]) &
    (df_aeonbukittinggi_excl_trans_type["casaname"] == df_diamond["casaname"].values[0]) &
    (df_aeonbukittinggi_excl_trans_type["state"] == df_diamond["state"].values[0]) &
    (df_aeonbukittinggi_excl_trans_type["Holiday"] == 0) &
    (df_aeonbukittinggi_excl_trans_type["date"] <= '2023-06-06')
    ]
    try:
      weight = round((df_diamond['sum_check_subtotal'].values[0] - df_gem['sum_check_subtotal'].values[0])/(df_gem['sum_check_subtotal'].values[0]), 4)
    except:
      weight = 0
    return(weight)

df_holiday_weights_aeonbukittinggi = pd.DataFrame(holidays_aeonbukittinggi, columns = ["holidays"])
df_holiday_weights_aeonbukittinggi["weights"] = df_holiday_weights_aeonbukittinggi["holidays"].apply(lambda x: generate_public_holiday_weights(x))

df_sales_by_day_aeonbukittinggi_excl_trans_type = pd.merge(df_aeonbukittinggi_excl_trans_type, df_holiday_weights_aeonbukittinggi, left_on = ['Holiday'], right_on = ["holidays"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_aeonbukittinggi_excl_trans_type = df_sales_by_day_aeonbukittinggi_excl_trans_type.drop(["holidays"], axis = 1)
df_sales_by_day_aeonbukittinggi_excl_trans_type = df_sales_by_day_aeonbukittinggi_excl_trans_type.replace(np.nan, 0)


def generate_public_holiday_weights(holiday_name):
    df_diamond = df_aeonbukittinggi_incl_trans_type_dinein[(df_aeonbukittinggi_incl_trans_type_dinein["Holiday"] == str(holiday_name)) & (df_aeonbukittinggi_incl_trans_type_dinein["date"] <= '2023-06-06')][["day", "weekend", "store_code", "casaname", "state", "sum_check_subtotal", "school_holidays_remark"]]
    df_gem = df_aeonbukittinggi_incl_trans_type_dinein[
    (df_aeonbukittinggi_incl_trans_type_dinein["day"] == df_diamond["day"].values[0]) &
    (df_aeonbukittinggi_incl_trans_type_dinein["weekend"] == df_diamond["weekend"].values[0]) &
    (df_aeonbukittinggi_incl_trans_type_dinein["store_code"] == df_diamond["store_code"].values[0]) &
    (df_aeonbukittinggi_incl_trans_type_dinein["casaname"] == df_diamond["casaname"].values[0]) &
    (df_aeonbukittinggi_incl_trans_type_dinein["state"] == df_diamond["state"].values[0]) &
    (df_aeonbukittinggi_incl_trans_type_dinein["Holiday"] == 0) &
    (df_aeonbukittinggi_incl_trans_type_dinein["date"] <= '2023-06-06')
    ]
    try:
      weight = round((df_diamond['sum_check_subtotal'].values[0] - df_gem['sum_check_subtotal'].values[0])/(df_gem['sum_check_subtotal'].values[0]), 4)
    except:
      weight = 0
    return(weight)

df_holiday_weights_aeonbukittinggi = pd.DataFrame(holidays_aeonbukittinggi, columns = ["holidays"])
df_holiday_weights_aeonbukittinggi["weights"] = df_holiday_weights_aeonbukittinggi["holidays"].apply(lambda x: generate_public_holiday_weights(x))

df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein = pd.merge(df_aeonbukittinggi_incl_trans_type_dinein, df_holiday_weights_aeonbukittinggi, left_on = ['Holiday'], right_on = ["holidays"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein = df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein.drop(["holidays"], axis = 1)
df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein = df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein.replace(np.nan, 0)


def generate_public_holiday_weights(holiday_name):
    df_diamond = df_aeonbukittinggi_incl_trans_type_takeaway[(df_aeonbukittinggi_incl_trans_type_takeaway["Holiday"] == str(holiday_name)) & (df_aeonbukittinggi_incl_trans_type_takeaway["date"] <= '2023-06-06')][["day", "weekend", "store_code", "casaname", "state", "sum_check_subtotal", "school_holidays_remark"]]
    df_gem = df_aeonbukittinggi_incl_trans_type_takeaway[
    (df_aeonbukittinggi_incl_trans_type_takeaway["day"] == df_diamond["day"].values[0]) &
    (df_aeonbukittinggi_incl_trans_type_takeaway["weekend"] == df_diamond["weekend"].values[0]) &
    (df_aeonbukittinggi_incl_trans_type_takeaway["store_code"] == df_diamond["store_code"].values[0]) &
    (df_aeonbukittinggi_incl_trans_type_takeaway["casaname"] == df_diamond["casaname"].values[0]) &
    (df_aeonbukittinggi_incl_trans_type_takeaway["state"] == df_diamond["state"].values[0]) &
    (df_aeonbukittinggi_incl_trans_type_takeaway["Holiday"] == 0) &
    (df_aeonbukittinggi_incl_trans_type_takeaway["date"] <= '2023-06-06')
    ]
    try:
      weight = round((df_diamond['sum_check_subtotal'].values[0] - df_gem['sum_check_subtotal'].values[0])/(df_gem['sum_check_subtotal'].values[0]), 4)
    except:
      weight = 0
    return(weight)

df_holiday_weights_aeonbukittinggi = pd.DataFrame(holidays_aeonbukittinggi, columns = ["holidays"])
df_holiday_weights_aeonbukittinggi["weights"] = df_holiday_weights_aeonbukittinggi["holidays"].apply(lambda x: generate_public_holiday_weights(x))

df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway = pd.merge(df_aeonbukittinggi_incl_trans_type_takeaway, df_holiday_weights_aeonbukittinggi, left_on = ['Holiday'], right_on = ["holidays"], how = 'left', suffixes = (None, '_y'))
df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway = df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway.drop(["holidays"], axis = 1)
df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway = df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway.replace(np.nan, 0)

# Existing daily forecast MAE and MAPE using gbr

In [36]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import HuberRegressor
from sklearn.model_selection import train_test_split

In [37]:
#1 UTAMA
df_sales_by_day_1utama_excl_trans_type_ohe = pd.get_dummies(df_sales_by_day_1utama_excl_trans_type, columns = ['day', 'Holiday'], dtype = float)
df_sales_by_day_1utama_excl_trans_type_ohe = df_sales_by_day_1utama_excl_trans_type_ohe.drop(columns = ["store_code", "casaname", "state", "sum_number_of_transactions", "sum_check_tax", "States"])

df_sales_by_day_1utama_excl_trans_type_ohe_train = df_sales_by_day_1utama_excl_trans_type_ohe[df_sales_by_day_1utama_excl_trans_type_ohe["date"] <= "2023-08-31"]
df_sales_by_day_1utama_excl_trans_type_ohe_test = df_sales_by_day_1utama_excl_trans_type_ohe[df_sales_by_day_1utama_excl_trans_type_ohe["date"] > "2023-08-31"]

X = list(df_sales_by_day_1utama_excl_trans_type_ohe_train.columns)
X.remove("sum_check_subtotal")
X.remove("date")

y = df_sales_by_day_1utama_excl_trans_type_ohe_train["sum_check_subtotal"]

X_df = df_sales_by_day_1utama_excl_trans_type_ohe_train[X]

x_train, x_test, y_train, y_test = train_test_split(X_df, y, test_size=0.15, random_state=42)

reg = GradientBoostingRegressor(random_state=0)

reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

#internal dataset test
df = pd.DataFrame()
df["y_pred"] = list(y_pred)
df["y_test"] = list(y_test)
df["difference"] = df["y_pred"] - df["y_test"]
df["abs_diff"] = df["difference"].abs()
df["pct_diff"] = (df["difference"]/df["y_test"]*100).abs()
print("Average MAE in internal test set is RM " + str(round(df["abs_diff"].mean(), 2)))
print("Average MAPE in internal test set is " + str(round(df["pct_diff"].mean(), 2)) + "%")

#external dataset test
df_sales_by_day_1utama_excl_trans_type_perf = pd.DataFrame(reg.predict(df_sales_by_day_1utama_excl_trans_type_ohe_test[X]), columns = ["predicted"])
df_sales_by_day_1utama_excl_trans_type_perf["actual"] = df_sales_by_day_1utama_excl_trans_type_ohe_test["sum_check_subtotal"].reset_index().drop(columns = ["index"])
df_sales_by_day_1utama_excl_trans_type_perf["abs_diff"] = (df_sales_by_day_1utama_excl_trans_type_perf["predicted"] - df_sales_by_day_1utama_excl_trans_type_perf["actual"]).abs()
df_sales_by_day_1utama_excl_trans_type_perf["pct_diff"] = (df_sales_by_day_1utama_excl_trans_type_perf["abs_diff"]/df_sales_by_day_1utama_excl_trans_type_perf["actual"]*100).abs()
print("Average MAE in external test set is RM " + str(round(df_sales_by_day_1utama_excl_trans_type_perf["abs_diff"].mean(), 2)))
print("Average MAPE in external test set is " + str(round(df_sales_by_day_1utama_excl_trans_type_perf["pct_diff"].mean(), 2)) + "%")

Average MAE in internal test set is RM 3296.99
Average MAPE in internal test set is 12.57%
Average MAE in external test set is RM 3147.22
Average MAPE in external test set is 11.96%


In [38]:
#AEON AU2
df_sales_by_day_aeonau2_excl_trans_type_ohe = pd.get_dummies(df_sales_by_day_aeonau2_excl_trans_type, columns = ['day', 'Holiday'], dtype = float)
df_sales_by_day_aeonau2_excl_trans_type_ohe = df_sales_by_day_aeonau2_excl_trans_type_ohe.drop(columns = ["store_code", "casaname", "state", "sum_number_of_transactions", "sum_check_tax", "States"])

df_sales_by_day_aeonau2_excl_trans_type_ohe_train = df_sales_by_day_aeonau2_excl_trans_type_ohe[df_sales_by_day_aeonau2_excl_trans_type_ohe["date"] <= "2023-08-31"]
df_sales_by_day_aeonau2_excl_trans_type_ohe_test = df_sales_by_day_aeonau2_excl_trans_type_ohe[df_sales_by_day_aeonau2_excl_trans_type_ohe["date"] > "2023-08-31"]

X = list(df_sales_by_day_aeonau2_excl_trans_type_ohe_train.columns)
X.remove("sum_check_subtotal")
X.remove("date")

y = df_sales_by_day_aeonau2_excl_trans_type_ohe_train["sum_check_subtotal"]

X_df = df_sales_by_day_aeonau2_excl_trans_type_ohe_train[X]

x_train, x_test, y_train, y_test = train_test_split(X_df, y, test_size=0.15, random_state=42)

reg = GradientBoostingRegressor(random_state=0)

reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

#internal dataset test
df = pd.DataFrame()
df["y_pred"] = list(y_pred)
df["y_test"] = list(y_test)
df["difference"] = df["y_pred"] - df["y_test"]
df["abs_diff"] = df["difference"].abs()
df["pct_diff"] = (df["difference"]/df["y_test"]*100).abs()
print("Average MAE in internal test set is RM " + str(round(df["abs_diff"].mean(), 2)))
print("Average MAPE in internal test set is " + str(round(df["pct_diff"].mean(), 2)) + "%")

#external dataset test
df_sales_by_day_aeonau2_excl_trans_type_perf = pd.DataFrame(reg.predict(df_sales_by_day_aeonau2_excl_trans_type_ohe_test[X]), columns = ["predicted"])
df_sales_by_day_aeonau2_excl_trans_type_perf["actual"] = df_sales_by_day_aeonau2_excl_trans_type_ohe_test["sum_check_subtotal"].reset_index().drop(columns = ["index"])
df_sales_by_day_aeonau2_excl_trans_type_perf["abs_diff"] = (df_sales_by_day_aeonau2_excl_trans_type_perf["predicted"] - df_sales_by_day_aeonau2_excl_trans_type_perf["actual"]).abs()
df_sales_by_day_aeonau2_excl_trans_type_perf["pct_diff"] = (df_sales_by_day_aeonau2_excl_trans_type_perf["abs_diff"]/df_sales_by_day_aeonau2_excl_trans_type_perf["actual"]*100).abs()
print("Average MAE in external test set is RM " + str(round(df_sales_by_day_aeonau2_excl_trans_type_perf["abs_diff"].mean(), 2)))
print("Average MAPE in external test set is " + str(round(df_sales_by_day_aeonau2_excl_trans_type_perf["pct_diff"].mean(), 2)) + "%")

Average MAE in internal test set is RM 1771.92
Average MAPE in internal test set is 21.87%
Average MAE in external test set is RM 1823.68
Average MAPE in external test set is 33.46%


In [39]:
#AEON BANDARAYA MELAKA
df_sales_by_day_aeonbandarayamelaka_excl_trans_type_ohe = pd.get_dummies(df_sales_by_day_aeonbandarayamelaka_excl_trans_type, columns = ['day', 'Holiday'], dtype = float)
df_sales_by_day_aeonbandarayamelaka_excl_trans_type_ohe = df_sales_by_day_aeonbandarayamelaka_excl_trans_type_ohe.drop(columns = ["store_code", "casaname", "state", "sum_number_of_transactions", "sum_check_tax", "States"])

df_sales_by_day_aeonbandarayamelaka_excl_trans_type_ohe_train = df_sales_by_day_aeonbandarayamelaka_excl_trans_type_ohe[df_sales_by_day_aeonbandarayamelaka_excl_trans_type_ohe["date"] <= "2023-08-31"]
df_sales_by_day_aeonbandarayamelaka_excl_trans_type_ohe_test = df_sales_by_day_aeonbandarayamelaka_excl_trans_type_ohe[df_sales_by_day_aeonbandarayamelaka_excl_trans_type_ohe["date"] > "2023-08-31"]

X = list(df_sales_by_day_aeonbandarayamelaka_excl_trans_type_ohe_train.columns)
X.remove("sum_check_subtotal")
X.remove("date")

y = df_sales_by_day_aeonbandarayamelaka_excl_trans_type_ohe_train["sum_check_subtotal"]

X_df = df_sales_by_day_aeonbandarayamelaka_excl_trans_type_ohe_train[X]

x_train, x_test, y_train, y_test = train_test_split(X_df, y, test_size=0.15, random_state=42)

reg = GradientBoostingRegressor(random_state=0)

reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

#internal dataset test
df = pd.DataFrame()
df["y_pred"] = list(y_pred)
df["y_test"] = list(y_test)
df["difference"] = df["y_pred"] - df["y_test"]
df["abs_diff"] = df["difference"].abs()
df["pct_diff"] = (df["difference"]/df["y_test"]*100).abs()
print("Average MAE in internal test set is RM " + str(round(df["abs_diff"].mean(), 2)))
print("Average MAPE in internal test set is " + str(round(df["pct_diff"].mean(), 2)) + "%")

#external dataset test
df_sales_by_day_aeonbandarayamelaka_excl_trans_type_perf = pd.DataFrame(reg.predict(df_sales_by_day_aeonbandarayamelaka_excl_trans_type_ohe_test[X]), columns = ["predicted"])
df_sales_by_day_aeonbandarayamelaka_excl_trans_type_perf["actual"] = df_sales_by_day_aeonbandarayamelaka_excl_trans_type_ohe_test["sum_check_subtotal"].reset_index().drop(columns = ["index"])
df_sales_by_day_aeonbandarayamelaka_excl_trans_type_perf["abs_diff"] = (df_sales_by_day_aeonbandarayamelaka_excl_trans_type_perf["predicted"] - df_sales_by_day_aeonbandarayamelaka_excl_trans_type_perf["actual"]).abs()
df_sales_by_day_aeonbandarayamelaka_excl_trans_type_perf["pct_diff"] = (df_sales_by_day_aeonbandarayamelaka_excl_trans_type_perf["abs_diff"]/df_sales_by_day_aeonbandarayamelaka_excl_trans_type_perf["actual"]*100).abs()
print("Average MAE in external test set is RM " + str(round(df_sales_by_day_aeonbandarayamelaka_excl_trans_type_perf["abs_diff"].mean(), 2)))
print("Average MAPE in external test set is " + str(round(df_sales_by_day_aeonbandarayamelaka_excl_trans_type_perf["pct_diff"].mean(), 2)) + "%")

Average MAE in internal test set is RM 3275.32
Average MAPE in internal test set is 14.62%
Average MAE in external test set is RM 3070.47
Average MAPE in external test set is 20.56%


In [40]:
#AEON BUKIT MERTAJAM
df_sales_by_day_aeonbukitmertajam_excl_trans_type_ohe = pd.get_dummies(df_sales_by_day_aeonbukitmertajam_excl_trans_type, columns = ['day', 'Holiday'], dtype = float)
df_sales_by_day_aeonbukitmertajam_excl_trans_type_ohe = df_sales_by_day_aeonbukitmertajam_excl_trans_type_ohe.drop(columns = ["store_code", "casaname", "state", "sum_number_of_transactions", "sum_check_tax", "States"])

df_sales_by_day_aeonbukitmertajam_excl_trans_type_ohe_train = df_sales_by_day_aeonbukitmertajam_excl_trans_type_ohe[df_sales_by_day_aeonbukitmertajam_excl_trans_type_ohe["date"] <= "2023-08-31"]
df_sales_by_day_aeonbukitmertajam_excl_trans_type_ohe_test = df_sales_by_day_aeonbukitmertajam_excl_trans_type_ohe[df_sales_by_day_aeonbukitmertajam_excl_trans_type_ohe["date"] > "2023-08-31"]

X = list(df_sales_by_day_aeonbukitmertajam_excl_trans_type_ohe_train.columns)
X.remove("sum_check_subtotal")
X.remove("date")

y = df_sales_by_day_aeonbukitmertajam_excl_trans_type_ohe_train["sum_check_subtotal"]

X_df = df_sales_by_day_aeonbukitmertajam_excl_trans_type_ohe_train[X]

x_train, x_test, y_train, y_test = train_test_split(X_df, y, test_size=0.15, random_state=42)

reg = GradientBoostingRegressor(random_state=0)

reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

#internal dataset test
df = pd.DataFrame()
df["y_pred"] = list(y_pred)
df["y_test"] = list(y_test)
df["difference"] = df["y_pred"] - df["y_test"]
df["abs_diff"] = df["difference"].abs()
df["pct_diff"] = (df["difference"]/df["y_test"]*100).abs()
print("Average MAE in internal test set is RM " + str(round(df["abs_diff"].mean(), 2)))
print("Average MAPE in internal test set is " + str(round(df["pct_diff"].mean(), 2)) + "%")

#external dataset test
df_sales_by_day_aeonbukitmertajam_excl_trans_type_perf = pd.DataFrame(reg.predict(df_sales_by_day_aeonbukitmertajam_excl_trans_type_ohe_test[X]), columns = ["predicted"])
df_sales_by_day_aeonbukitmertajam_excl_trans_type_perf["actual"] = df_sales_by_day_aeonbukitmertajam_excl_trans_type_ohe_test["sum_check_subtotal"].reset_index().drop(columns = ["index"])
df_sales_by_day_aeonbukitmertajam_excl_trans_type_perf["abs_diff"] = (df_sales_by_day_aeonbukitmertajam_excl_trans_type_perf["predicted"] - df_sales_by_day_aeonbukitmertajam_excl_trans_type_perf["actual"]).abs()
df_sales_by_day_aeonbukitmertajam_excl_trans_type_perf["pct_diff"] = (df_sales_by_day_aeonbukitmertajam_excl_trans_type_perf["abs_diff"]/df_sales_by_day_aeonbukitmertajam_excl_trans_type_perf["actual"]*100).abs()
print("Average MAE in external test set is RM " + str(round(df_sales_by_day_aeonbukitmertajam_excl_trans_type_perf["abs_diff"].mean(), 2)))
print("Average MAPE in external test set is " + str(round(df_sales_by_day_aeonbukitmertajam_excl_trans_type_perf["pct_diff"].mean(), 2)) + "%")

Average MAE in internal test set is RM 2570.61
Average MAPE in internal test set is 22.46%
Average MAE in external test set is RM 2663.27
Average MAPE in external test set is 34.65%


In [41]:
#AEON BUKIT TINGGI
df_sales_by_day_aeonbukittinggi_excl_trans_type_ohe = pd.get_dummies(df_sales_by_day_aeonbukittinggi_excl_trans_type, columns = ['day', 'Holiday'], dtype = float)
df_sales_by_day_aeonbukittinggi_excl_trans_type_ohe = df_sales_by_day_aeonbukittinggi_excl_trans_type_ohe.drop(columns = ["store_code", "casaname", "state", "sum_number_of_transactions", "sum_check_tax", "States"])

df_sales_by_day_aeonbukittinggi_excl_trans_type_ohe_train = df_sales_by_day_aeonbukittinggi_excl_trans_type_ohe[df_sales_by_day_aeonbukittinggi_excl_trans_type_ohe["date"] <= "2023-08-31"]
df_sales_by_day_aeonbukittinggi_excl_trans_type_ohe_test = df_sales_by_day_aeonbukittinggi_excl_trans_type_ohe[df_sales_by_day_aeonbukittinggi_excl_trans_type_ohe["date"] > "2023-08-31"]

X = list(df_sales_by_day_aeonbukittinggi_excl_trans_type_ohe_train.columns)
X.remove("sum_check_subtotal")
X.remove("date")

y = df_sales_by_day_aeonbukittinggi_excl_trans_type_ohe_train["sum_check_subtotal"]

X_df = df_sales_by_day_aeonbukittinggi_excl_trans_type_ohe_train[X]

x_train, x_test, y_train, y_test = train_test_split(X_df, y, test_size=0.15, random_state=42)

reg = GradientBoostingRegressor(random_state=0)

reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

#internal dataset test
df = pd.DataFrame()
df["y_pred"] = list(y_pred)
df["y_test"] = list(y_test)
df["difference"] = df["y_pred"] - df["y_test"]
df["abs_diff"] = df["difference"].abs()
df["pct_diff"] = (df["difference"]/df["y_test"]*100).abs()
print("Average MAE in internal test set is RM " + str(round(df["abs_diff"].mean(), 2)))
print("Average MAPE in internal test set is " + str(round(df["pct_diff"].mean(), 2)) + "%")

#external dataset test
df_sales_by_day_aeonbukittinggi_excl_trans_type_perf = pd.DataFrame(reg.predict(df_sales_by_day_aeonbukittinggi_excl_trans_type_ohe_test[X]), columns = ["predicted"])
df_sales_by_day_aeonbukittinggi_excl_trans_type_perf["actual"] = df_sales_by_day_aeonbukittinggi_excl_trans_type_ohe_test["sum_check_subtotal"].reset_index().drop(columns = ["index"])
df_sales_by_day_aeonbukittinggi_excl_trans_type_perf["abs_diff"] = (df_sales_by_day_aeonbukittinggi_excl_trans_type_perf["predicted"] - df_sales_by_day_aeonbukittinggi_excl_trans_type_perf["actual"]).abs()
df_sales_by_day_aeonbukittinggi_excl_trans_type_perf["pct_diff"] = (df_sales_by_day_aeonbukittinggi_excl_trans_type_perf["abs_diff"]/df_sales_by_day_aeonbukittinggi_excl_trans_type_perf["actual"]*100).abs()
print("Average MAE in external test set is RM " + str(round(df_sales_by_day_aeonbukittinggi_excl_trans_type_perf["abs_diff"].mean(), 2)))
print("Average MAPE in external test set is " + str(round(df_sales_by_day_aeonbukittinggi_excl_trans_type_perf["pct_diff"].mean(), 2)) + "%")

Average MAE in internal test set is RM 2234.51
Average MAPE in internal test set is 17.37%
Average MAE in external test set is RM 3268.22
Average MAPE in external test set is 33.39%


# Updated daily forecast (dine-in and takeaway) MAE and MAPE using gbr

In [116]:
#dine-in: test OPR variable
df_sales_by_day_1utama_incl_trans_type_dinein_econ = pd.merge(df_sales_by_day_1utama_incl_trans_type_dinein, df_opr_data[["date", "new_opr_level"]], left_on = ["date"], right_on = ["date"], how = "left")
df_sales_by_day_1utama_incl_trans_type_dinein_econ["new_opr_level"] = df_sales_by_day_1utama_incl_trans_type_dinein_econ["new_opr_level"].ffill()
df_sales_by_day_1utama_incl_trans_type_dinein_econ["new_opr_level"] = df_sales_by_day_1utama_incl_trans_type_dinein_econ["new_opr_level"].replace(np.nan, 1.75)

df_sales_by_day_aeonau2_incl_trans_type_dinein_econ = pd.merge(df_sales_by_day_aeonau2_incl_trans_type_dinein, df_opr_data[["date", "new_opr_level"]], left_on = ["date"], right_on = ["date"], how = "left")
df_sales_by_day_aeonau2_incl_trans_type_dinein_econ["new_opr_level"] = df_sales_by_day_aeonau2_incl_trans_type_dinein_econ["new_opr_level"].ffill()
df_sales_by_day_aeonau2_incl_trans_type_dinein_econ["new_opr_level"] = df_sales_by_day_aeonau2_incl_trans_type_dinein_econ["new_opr_level"].replace(np.nan, 1.75)

df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_econ = pd.merge(df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein, df_opr_data[["date", "new_opr_level"]], left_on = ["date"], right_on = ["date"], how = "left")
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_econ["new_opr_level"] = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_econ["new_opr_level"].ffill()
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_econ["new_opr_level"] = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_econ["new_opr_level"].replace(np.nan, 1.75)

df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_econ = pd.merge(df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein, df_opr_data[["date", "new_opr_level"]], left_on = ["date"], right_on = ["date"], how = "left")
df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_econ["new_opr_level"] = df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_econ["new_opr_level"].ffill()
df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_econ["new_opr_level"] = df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_econ["new_opr_level"].replace(np.nan, 1.75)

df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_econ = pd.merge(df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein, df_opr_data[["date", "new_opr_level"]], left_on = ["date"], right_on = ["date"], how = "left")
df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_econ["new_opr_level"] = df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_econ["new_opr_level"].ffill()
df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_econ["new_opr_level"] = df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_econ["new_opr_level"].replace(np.nan, 1.75)

#take-away: test OPR variable
df_sales_by_day_1utama_incl_trans_type_takeaway_econ = pd.merge(df_sales_by_day_1utama_incl_trans_type_takeaway, df_opr_data[["date", "new_opr_level"]], left_on = ["date"], right_on = ["date"], how = "left")
df_sales_by_day_1utama_incl_trans_type_takeaway_econ["new_opr_level"] = df_sales_by_day_1utama_incl_trans_type_takeaway_econ["new_opr_level"].ffill()
df_sales_by_day_1utama_incl_trans_type_takeaway_econ["new_opr_level"] = df_sales_by_day_1utama_incl_trans_type_takeaway_econ["new_opr_level"].replace(np.nan, 1.75)

df_sales_by_day_aeonau2_incl_trans_type_takeaway_econ = pd.merge(df_sales_by_day_aeonau2_incl_trans_type_takeaway, df_opr_data[["date", "new_opr_level"]], left_on = ["date"], right_on = ["date"], how = "left")
df_sales_by_day_aeonau2_incl_trans_type_takeaway_econ["new_opr_level"] = df_sales_by_day_aeonau2_incl_trans_type_takeaway_econ["new_opr_level"].ffill()
df_sales_by_day_aeonau2_incl_trans_type_takeaway_econ["new_opr_level"] = df_sales_by_day_aeonau2_incl_trans_type_takeaway_econ["new_opr_level"].replace(np.nan, 1.75)

df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_econ = pd.merge(df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway, df_opr_data[["date", "new_opr_level"]], left_on = ["date"], right_on = ["date"], how = "left")
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_econ["new_opr_level"] = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_econ["new_opr_level"].ffill()
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_econ["new_opr_level"] = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_econ["new_opr_level"].replace(np.nan, 1.75)

df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_econ = pd.merge(df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway, df_opr_data[["date", "new_opr_level"]], left_on = ["date"], right_on = ["date"], how = "left")
df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_econ["new_opr_level"] = df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_econ["new_opr_level"].ffill()
df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_econ["new_opr_level"] = df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_econ["new_opr_level"].replace(np.nan, 1.75)

df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_econ = pd.merge(df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway, df_opr_data[["date", "new_opr_level"]], left_on = ["date"], right_on = ["date"], how = "left")
df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_econ["new_opr_level"] = df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_econ["new_opr_level"].ffill()
df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_econ["new_opr_level"] = df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_econ["new_opr_level"].replace(np.nan, 1.75)

In [117]:
#1 UTAMA: dine-in
df_sales_by_day_1utama_incl_trans_type_dinein_ohe = pd.get_dummies(df_sales_by_day_1utama_incl_trans_type_dinein_econ, columns = ['day', 'Holiday'], dtype = float)
df_sales_by_day_1utama_incl_trans_type_dinein_ohe = df_sales_by_day_1utama_incl_trans_type_dinein_ohe.drop(columns = ["store_code", "casaname", "state", "trans_type_int", "trans_type", "sum_number_of_transactions", "sum_check_tax", "States"])

df_sales_by_day_1utama_incl_trans_type_dinein_ohe_train = df_sales_by_day_1utama_incl_trans_type_dinein_ohe[df_sales_by_day_1utama_incl_trans_type_dinein_ohe["date"] <= "2023-08-31"]
df_sales_by_day_1utama_incl_trans_type_dinein_ohe_test = df_sales_by_day_1utama_incl_trans_type_dinein_ohe[df_sales_by_day_1utama_incl_trans_type_dinein_ohe["date"] > "2023-08-31"]

X = list(df_sales_by_day_1utama_incl_trans_type_dinein_ohe_train.columns)
X.remove("sum_check_subtotal")
X.remove("date")

y = df_sales_by_day_1utama_incl_trans_type_dinein_ohe_train["sum_check_subtotal"]

X_df = df_sales_by_day_1utama_incl_trans_type_dinein_ohe_train[X]

x_train, x_test, y_train, y_test = train_test_split(X_df, y, test_size=0.15, random_state=42)

reg = GradientBoostingRegressor(random_state=0)

reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

#internal dataset test
df = pd.DataFrame()
df["y_pred"] = list(y_pred)
df["y_test"] = list(y_test)
df["difference"] = df["y_pred"] - df["y_test"]
df["abs_diff"] = df["difference"].abs()
df["pct_diff"] = (df["difference"]/df["y_test"]*100).abs()
print("Average MAE in internal test set is RM " + str(round(df["abs_diff"].mean(), 2)))
print("Average MAPE in internal test set is " + str(round(df["pct_diff"].mean(), 2)) + "%")

#external dataset test
df_sales_by_day_1utama_incl_trans_type_dinein_perf = pd.DataFrame(reg.predict(df_sales_by_day_1utama_incl_trans_type_dinein_ohe_test[X]), columns = ["predicted"])
df_sales_by_day_1utama_incl_trans_type_dinein_perf["actual"] = df_sales_by_day_1utama_incl_trans_type_dinein_ohe_test["sum_check_subtotal"].reset_index().drop(columns = ["index"])
df_sales_by_day_1utama_incl_trans_type_dinein_perf["abs_diff"] = (df_sales_by_day_1utama_incl_trans_type_dinein_perf["predicted"] - df_sales_by_day_1utama_incl_trans_type_dinein_perf["actual"]).abs()
df_sales_by_day_1utama_incl_trans_type_dinein_perf["pct_diff"] = (df_sales_by_day_1utama_incl_trans_type_dinein_perf["abs_diff"]/df_sales_by_day_1utama_incl_trans_type_dinein_perf["actual"]*100).abs()
print("Average MAE in external test set is RM " + str(round(df_sales_by_day_1utama_incl_trans_type_dinein_perf["abs_diff"].mean(), 2)))
print("Average MAPE in external test set is " + str(round(df_sales_by_day_1utama_incl_trans_type_dinein_perf["pct_diff"].mean(), 2)) + "%")

Average MAE in internal test set is RM 2059.55
Average MAPE in internal test set is 8.88%
Average MAE in external test set is RM 2578.67
Average MAPE in external test set is 12.83%


In [118]:
#AEON AU2: dine-in
df_sales_by_day_aeonau2_incl_trans_type_dinein_ohe = pd.get_dummies(df_sales_by_day_aeonau2_incl_trans_type_dinein_econ, columns = ['day', 'Holiday'], dtype = float)
df_sales_by_day_aeonau2_incl_trans_type_dinein_ohe = df_sales_by_day_aeonau2_incl_trans_type_dinein_ohe.drop(columns = ["store_code", "casaname", "state", "trans_type_int", "trans_type", "sum_number_of_transactions", "sum_check_tax", "States"])

df_sales_by_day_aeonau2_incl_trans_type_dinein_ohe_train = df_sales_by_day_aeonau2_incl_trans_type_dinein_ohe[df_sales_by_day_aeonau2_incl_trans_type_dinein_ohe["date"] <= "2023-08-31"]
df_sales_by_day_aeonau2_incl_trans_type_dinein_ohe_test = df_sales_by_day_aeonau2_incl_trans_type_dinein_ohe[df_sales_by_day_aeonau2_incl_trans_type_dinein_ohe["date"] > "2023-08-31"]

X = list(df_sales_by_day_aeonau2_incl_trans_type_dinein_ohe_train.columns)
X.remove("sum_check_subtotal")
X.remove("date")

y = df_sales_by_day_aeonau2_incl_trans_type_dinein_ohe_train["sum_check_subtotal"]

X_df = df_sales_by_day_aeonau2_incl_trans_type_dinein_ohe_train[X]

x_train, x_test, y_train, y_test = train_test_split(X_df, y, test_size=0.15, random_state=42)

reg = GradientBoostingRegressor(random_state=0)

reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

#internal dataset test
df = pd.DataFrame()
df["y_pred"] = list(y_pred)
df["y_test"] = list(y_test)
df["difference"] = df["y_pred"] - df["y_test"]
df["abs_diff"] = df["difference"].abs()
df["pct_diff"] = (df["difference"]/df["y_test"]*100).abs()
print("Average MAE in internal test set is RM " + str(round(df["abs_diff"].mean(), 2)))
print("Average MAPE in internal test set is " + str(round(df["pct_diff"].mean(), 2)) + "%")

#external dataset test
df_sales_by_day_aeonau2_incl_trans_type_dinein_perf = pd.DataFrame(reg.predict(df_sales_by_day_aeonau2_incl_trans_type_dinein_ohe_test[X]), columns = ["predicted"])
df_sales_by_day_aeonau2_incl_trans_type_dinein_perf["actual"] = df_sales_by_day_aeonau2_incl_trans_type_dinein_ohe_test["sum_check_subtotal"].reset_index().drop(columns = ["index"])
df_sales_by_day_aeonau2_incl_trans_type_dinein_perf["abs_diff"] = (df_sales_by_day_aeonau2_incl_trans_type_dinein_perf["predicted"] - df_sales_by_day_aeonau2_incl_trans_type_dinein_perf["actual"]).abs()
df_sales_by_day_aeonau2_incl_trans_type_dinein_perf["pct_diff"] = (df_sales_by_day_aeonau2_incl_trans_type_dinein_perf["abs_diff"]/df_sales_by_day_aeonau2_incl_trans_type_dinein_perf["actual"]*100).abs()
print("Average MAE in external test set is RM " + str(round(df_sales_by_day_aeonau2_incl_trans_type_dinein_perf["abs_diff"].mean(), 2)))
print("Average MAPE in external test set is " + str(round(df_sales_by_day_aeonau2_incl_trans_type_dinein_perf["pct_diff"].mean(), 2)) + "%")

Average MAE in internal test set is RM 1178.29
Average MAPE in internal test set is 26.18%
Average MAE in external test set is RM 1087.31
Average MAPE in external test set is 28.8%


In [119]:
#AEON BANDARAYA MELAKA: dine-in
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_ohe = pd.get_dummies(df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_econ, columns = ['day', 'Holiday'], dtype = float)
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_ohe = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_ohe.drop(columns = ["store_code", "casaname", "state", "trans_type_int", "trans_type", "sum_number_of_transactions", "sum_check_tax", "States"])

df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_ohe_train = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_ohe[df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_ohe["date"] <= "2023-08-31"]
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_ohe_test = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_ohe[df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_ohe["date"] > "2023-08-31"]

X = list(df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_ohe_train.columns)
X.remove("sum_check_subtotal")
X.remove("date")

y = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_ohe_train["sum_check_subtotal"]

X_df = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_ohe_train[X]

x_train, x_test, y_train, y_test = train_test_split(X_df, y, test_size=0.15, random_state=42)

reg = GradientBoostingRegressor(random_state=0)

reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

#internal dataset test
df = pd.DataFrame()
df["y_pred"] = list(y_pred)
df["y_test"] = list(y_test)
df["difference"] = df["y_pred"] - df["y_test"]
df["abs_diff"] = df["difference"].abs()
df["pct_diff"] = (df["difference"]/df["y_test"]*100).abs()
print("Average MAE in internal test set is RM " + str(round(df["abs_diff"].mean(), 2)))
print("Average MAPE in internal test set is " + str(round(df["pct_diff"].mean(), 2)) + "%")

#external dataset test
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_perf = pd.DataFrame(reg.predict(df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_ohe_test[X]), columns = ["predicted"])
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_perf["actual"] = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_ohe_test["sum_check_subtotal"].reset_index().drop(columns = ["index"])
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_perf["abs_diff"] = (df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_perf["predicted"] - df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_perf["actual"]).abs()
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_perf["pct_diff"] = (df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_perf["abs_diff"]/df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_perf["actual"]*100).abs()
print("Average MAE in external test set is RM " + str(round(df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_perf["abs_diff"].mean(), 2)))
print("Average MAPE in external test set is " + str(round(df_sales_by_day_aeonbandarayamelaka_incl_trans_type_dinein_perf["pct_diff"].mean(), 2)) + "%")

Average MAE in internal test set is RM 1951.99
Average MAPE in internal test set is 12.47%
Average MAE in external test set is RM 2106.96
Average MAPE in external test set is 15.25%


In [120]:
#AEON BUKIT MERTAJAM: dine-in
df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_ohe = pd.get_dummies(df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_econ, columns = ['day', 'Holiday'], dtype = float)
df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_ohe = df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_ohe.drop(columns = ["store_code", "casaname", "state", "trans_type_int", "trans_type", "sum_number_of_transactions", "sum_check_tax", "States"])

df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_ohe_train = df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_ohe[df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_ohe["date"] <= "2023-08-31"]
df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_ohe_test = df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_ohe[df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_ohe["date"] > "2023-08-31"]

X = list(df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_ohe_train.columns)
X.remove("sum_check_subtotal")
X.remove("date")

y = df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_ohe_train["sum_check_subtotal"]

X_df = df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_ohe_train[X]

x_train, x_test, y_train, y_test = train_test_split(X_df, y, test_size=0.15, random_state=42)

reg = GradientBoostingRegressor(random_state=0)

reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

#internal dataset test
df = pd.DataFrame()
df["y_pred"] = list(y_pred)
df["y_test"] = list(y_test)
df["difference"] = df["y_pred"] - df["y_test"]
df["abs_diff"] = df["difference"].abs()
df["pct_diff"] = (df["difference"]/df["y_test"]*100).abs()
print("Average MAE in internal test set is RM " + str(round(df["abs_diff"].mean(), 2)))
print("Average MAPE in internal test set is " + str(round(df["pct_diff"].mean(), 2)) + "%")

#external dataset test
df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_perf = pd.DataFrame(reg.predict(df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_ohe_test[X]), columns = ["predicted"])
df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_perf["actual"] = df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_ohe_test["sum_check_subtotal"].reset_index().drop(columns = ["index"])
df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_perf["abs_diff"] = (df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_perf["predicted"] - df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_perf["actual"]).abs()
df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_perf["pct_diff"] = (df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_perf["abs_diff"]/df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_perf["actual"]*100).abs()
print("Average MAE in external test set is RM " + str(round(df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_perf["abs_diff"].mean(), 2)))
print("Average MAPE in external test set is " + str(round(df_sales_by_day_aeonbukitmertajam_incl_trans_type_dinein_perf["pct_diff"].mean(), 2)) + "%")

Average MAE in internal test set is RM 1422.62
Average MAPE in internal test set is 16.13%
Average MAE in external test set is RM 1444.41
Average MAPE in external test set is 22.04%


In [121]:
#AEON BUKIT TINGGI: dine-in
df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_ohe = pd.get_dummies(df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_econ, columns = ['day', 'Holiday'], dtype = float)
df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_ohe = df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_ohe.drop(columns = ["store_code", "casaname", "state", "trans_type_int", "trans_type", "sum_number_of_transactions", "sum_check_tax", "States"])

df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_ohe_train = df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_ohe[df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_ohe["date"] <= "2023-08-31"]
df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_ohe_test = df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_ohe[df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_ohe["date"] > "2023-08-31"]

X = list(df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_ohe_train.columns)
X.remove("sum_check_subtotal")
X.remove("date")

y = df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_ohe_train["sum_check_subtotal"]

X_df = df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_ohe_train[X]

x_train, x_test, y_train, y_test = train_test_split(X_df, y, test_size=0.15, random_state=42)

reg = GradientBoostingRegressor(random_state=0)

reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

#internal dataset test
df = pd.DataFrame()
df["y_pred"] = list(y_pred)
df["y_test"] = list(y_test)
df["difference"] = df["y_pred"] - df["y_test"]
df["abs_diff"] = df["difference"].abs()
df["pct_diff"] = (df["difference"]/df["y_test"]*100).abs()
print("Average MAE in internal test set is RM " + str(round(df["abs_diff"].mean(), 2)))
print("Average MAPE in internal test set is " + str(round(df["pct_diff"].mean(), 2)) + "%")

#external dataset test
df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_perf = pd.DataFrame(reg.predict(df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_ohe_test[X]), columns = ["predicted"])
df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_perf["actual"] = df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_ohe_test["sum_check_subtotal"].reset_index().drop(columns = ["index"])
df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_perf["abs_diff"] = (df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_perf["predicted"] - df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_perf["actual"]).abs()
df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_perf["pct_diff"] = (df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_perf["abs_diff"]/df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_perf["actual"]*100).abs()
print("Average MAE in external test set is RM " + str(round(df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_perf["abs_diff"].mean(), 2)))
print("Average MAPE in external test set is " + str(round(df_sales_by_day_aeonbukittinggi_incl_trans_type_dinein_perf["pct_diff"].mean(), 2)) + "%")

Average MAE in internal test set is RM 1433.32
Average MAPE in internal test set is 16.39%
Average MAE in external test set is RM 1393.32
Average MAPE in external test set is 18.23%


In [122]:
#1 UTAMA: take-away
df_sales_by_day_1utama_incl_trans_type_takeaway_ohe = pd.get_dummies(df_sales_by_day_1utama_incl_trans_type_takeaway_econ, columns = ['day', 'Holiday'], dtype = float)
df_sales_by_day_1utama_incl_trans_type_takeaway_ohe = df_sales_by_day_1utama_incl_trans_type_takeaway_ohe.drop(columns = ["store_code", "casaname", "state", "trans_type_int", "trans_type", "sum_number_of_transactions", "sum_check_tax", "States"])

df_sales_by_day_1utama_incl_trans_type_takeaway_ohe_train = df_sales_by_day_1utama_incl_trans_type_takeaway_ohe[df_sales_by_day_1utama_incl_trans_type_takeaway_ohe["date"] <= "2023-08-31"]
df_sales_by_day_1utama_incl_trans_type_takeaway_ohe_test = df_sales_by_day_1utama_incl_trans_type_takeaway_ohe[df_sales_by_day_1utama_incl_trans_type_takeaway_ohe["date"] > "2023-08-31"]

X = list(df_sales_by_day_1utama_incl_trans_type_takeaway_ohe_train.columns)
X.remove("sum_check_subtotal")
X.remove("date")

y = df_sales_by_day_1utama_incl_trans_type_takeaway_ohe_train["sum_check_subtotal"]

X_df = df_sales_by_day_1utama_incl_trans_type_takeaway_ohe_train[X]

x_train, x_test, y_train, y_test = train_test_split(X_df, y, test_size=0.15, random_state=42)

reg = GradientBoostingRegressor(random_state=0)

reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

#internal dataset test
df = pd.DataFrame()
df["y_pred"] = list(y_pred)
df["y_test"] = list(y_test)
df["difference"] = df["y_pred"] - df["y_test"]
df["abs_diff"] = df["difference"].abs()
df["pct_diff"] = (df["difference"]/df["y_test"]*100).abs()
print("Average MAE in internal test set is RM " + str(round(df["abs_diff"].mean(), 2)))
print("Average MAPE in internal test set is " + str(round(df["pct_diff"].mean(), 2)) + "%")

#external dataset test
df_sales_by_day_1utama_incl_trans_type_takeaway_perf = pd.DataFrame(reg.predict(df_sales_by_day_1utama_incl_trans_type_takeaway_ohe_test[X]), columns = ["predicted"])
df_sales_by_day_1utama_incl_trans_type_takeaway_perf["actual"] = df_sales_by_day_1utama_incl_trans_type_takeaway_ohe_test["sum_check_subtotal"].reset_index().drop(columns = ["index"])
df_sales_by_day_1utama_incl_trans_type_takeaway_perf["abs_diff"] = (df_sales_by_day_1utama_incl_trans_type_takeaway_perf["predicted"] - df_sales_by_day_1utama_incl_trans_type_takeaway_perf["actual"]).abs()
df_sales_by_day_1utama_incl_trans_type_takeaway_perf["pct_diff"] = (df_sales_by_day_1utama_incl_trans_type_takeaway_perf["abs_diff"]/df_sales_by_day_1utama_incl_trans_type_takeaway_perf["actual"]*100).abs()
print("Average MAE in external test set is RM " + str(round(df_sales_by_day_1utama_incl_trans_type_takeaway_perf["abs_diff"].mean(), 2)))
print("Average MAPE in external test set is " + str(round(df_sales_by_day_1utama_incl_trans_type_takeaway_perf["pct_diff"].mean(), 2)) + "%")

Average MAE in internal test set is RM 410.56
Average MAPE in internal test set is 35.97%
Average MAE in external test set is RM 358.32
Average MAPE in external test set is 27.4%


In [123]:
#AEON AU2: take-away
df_sales_by_day_aeonau2_incl_trans_type_takeaway_ohe = pd.get_dummies(df_sales_by_day_aeonau2_incl_trans_type_takeaway_econ, columns = ['day', 'Holiday'], dtype = float)
df_sales_by_day_aeonau2_incl_trans_type_takeaway_ohe = df_sales_by_day_aeonau2_incl_trans_type_takeaway_ohe.drop(columns = ["store_code", "casaname", "state", "trans_type_int", "trans_type", "sum_number_of_transactions", "sum_check_tax", "States"])

df_sales_by_day_aeonau2_incl_trans_type_takeaway_ohe_train = df_sales_by_day_aeonau2_incl_trans_type_takeaway_ohe[df_sales_by_day_aeonau2_incl_trans_type_takeaway_ohe["date"] <= "2023-08-31"]
df_sales_by_day_aeonau2_incl_trans_type_takeaway_ohe_test = df_sales_by_day_aeonau2_incl_trans_type_takeaway_ohe[df_sales_by_day_aeonau2_incl_trans_type_takeaway_ohe["date"] > "2023-08-31"]

X = list(df_sales_by_day_aeonau2_incl_trans_type_takeaway_ohe_train.columns)
X.remove("sum_check_subtotal")
X.remove("date")

y = df_sales_by_day_aeonau2_incl_trans_type_takeaway_ohe_train["sum_check_subtotal"]

X_df = df_sales_by_day_aeonau2_incl_trans_type_takeaway_ohe_train[X]

x_train, x_test, y_train, y_test = train_test_split(X_df, y, test_size=0.15, random_state=42)

reg = GradientBoostingRegressor(random_state=0)

reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

#internal dataset test
df = pd.DataFrame()
df["y_pred"] = list(y_pred)
df["y_test"] = list(y_test)
df["difference"] = df["y_pred"] - df["y_test"]
df["abs_diff"] = df["difference"].abs()
df["pct_diff"] = (df["difference"]/df["y_test"]*100).abs()
print("Average MAE in internal test set is RM " + str(round(df["abs_diff"].mean(), 2)))
print("Average MAPE in internal test set is " + str(round(df["pct_diff"].mean(), 2)) + "%")

#external dataset test
df_sales_by_day_aeonau2_incl_trans_type_takeaway_perf = pd.DataFrame(reg.predict(df_sales_by_day_aeonau2_incl_trans_type_takeaway_ohe_test[X]), columns = ["predicted"])
df_sales_by_day_aeonau2_incl_trans_type_takeaway_perf["actual"] = df_sales_by_day_aeonau2_incl_trans_type_takeaway_ohe_test["sum_check_subtotal"].reset_index().drop(columns = ["index"])
df_sales_by_day_aeonau2_incl_trans_type_takeaway_perf["abs_diff"] = (df_sales_by_day_aeonau2_incl_trans_type_takeaway_perf["predicted"] - df_sales_by_day_aeonau2_incl_trans_type_takeaway_perf["actual"]).abs()
df_sales_by_day_aeonau2_incl_trans_type_takeaway_perf["pct_diff"] = (df_sales_by_day_aeonau2_incl_trans_type_takeaway_perf["abs_diff"]/df_sales_by_day_aeonau2_incl_trans_type_takeaway_perf["actual"]*100).abs()
print("Average MAE in external test set is RM " + str(round(df_sales_by_day_aeonau2_incl_trans_type_takeaway_perf["abs_diff"].mean(), 2)))
print("Average MAPE in external test set is " + str(round(df_sales_by_day_aeonau2_incl_trans_type_takeaway_perf["pct_diff"].mean(), 2)) + "%")

Average MAE in internal test set is RM 333.44
Average MAPE in internal test set is 84.26%
Average MAE in external test set is RM 286.64
Average MAPE in external test set is 120.05%


In [124]:
#AEON BANDARAYA MELAKA: take-away
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_ohe = pd.get_dummies(df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_econ, columns = ['day', 'Holiday'], dtype = float)
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_ohe = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_ohe.drop(columns = ["store_code", "casaname", "state", "trans_type_int", "trans_type", "sum_number_of_transactions", "sum_check_tax", "States"])

df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_ohe_train = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_ohe[df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_ohe["date"] <= "2023-08-31"]
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_ohe_test = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_ohe[df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_ohe["date"] > "2023-08-31"]

X = list(df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_ohe_train.columns)
X.remove("sum_check_subtotal")
X.remove("date")

y = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_ohe_train["sum_check_subtotal"]

X_df = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_ohe_train[X]

x_train, x_test, y_train, y_test = train_test_split(X_df, y, test_size=0.15, random_state=42)

reg = GradientBoostingRegressor(random_state=0)

reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

#internal dataset test
df = pd.DataFrame()
df["y_pred"] = list(y_pred)
df["y_test"] = list(y_test)
df["difference"] = df["y_pred"] - df["y_test"]
df["abs_diff"] = df["difference"].abs()
df["pct_diff"] = (df["difference"]/df["y_test"]*100).abs()
print("Average MAE in internal test set is RM " + str(round(df["abs_diff"].mean(), 2)))
print("Average MAPE in internal test set is " + str(round(df["pct_diff"].mean(), 2)) + "%")

#external dataset test
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_perf = pd.DataFrame(reg.predict(df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_ohe_test[X]), columns = ["predicted"])
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_perf["actual"] = df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_ohe_test["sum_check_subtotal"].reset_index().drop(columns = ["index"])
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_perf["abs_diff"] = (df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_perf["predicted"] - df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_perf["actual"]).abs()
df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_perf["pct_diff"] = (df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_perf["abs_diff"]/df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_perf["actual"]*100).abs()
print("Average MAE in external test set is RM " + str(round(df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_perf["abs_diff"].mean(), 2)))
print("Average MAPE in external test set is " + str(round(df_sales_by_day_aeonbandarayamelaka_incl_trans_type_takeaway_perf["pct_diff"].mean(), 2)) + "%")

Average MAE in internal test set is RM 482.65
Average MAPE in internal test set is 52.77%
Average MAE in external test set is RM 355.73
Average MAPE in external test set is 108.29%


In [125]:
#AEON BUKIT MERTAJAM: take-away
df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_ohe = pd.get_dummies(df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_econ, columns = ['day', 'Holiday'], dtype = float)
df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_ohe = df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_ohe.drop(columns = ["store_code", "casaname", "state", "trans_type_int", "trans_type", "sum_number_of_transactions", "sum_check_tax", "States"])

df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_ohe_train = df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_ohe[df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_ohe["date"] <= "2023-08-31"]
df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_ohe_test = df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_ohe[df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_ohe["date"] > "2023-08-31"]

X = list(df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_ohe_train.columns)
X.remove("sum_check_subtotal")
X.remove("date")

y = df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_ohe_train["sum_check_subtotal"]

X_df = df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_ohe_train[X]

x_train, x_test, y_train, y_test = train_test_split(X_df, y, test_size=0.15, random_state=42)

reg = GradientBoostingRegressor(random_state=0)

reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

#internal dataset test
df = pd.DataFrame()
df["y_pred"] = list(y_pred)
df["y_test"] = list(y_test)
df["difference"] = df["y_pred"] - df["y_test"]
df["abs_diff"] = df["difference"].abs()
df["pct_diff"] = (df["difference"]/df["y_test"]*100).abs()
print("Average MAE in internal test set is RM " + str(round(df["abs_diff"].mean(), 2)))
print("Average MAPE in internal test set is " + str(round(df["pct_diff"].mean(), 2)) + "%")

#external dataset test
df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_perf = pd.DataFrame(reg.predict(df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_ohe_test[X]), columns = ["predicted"])
df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_perf["actual"] = df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_ohe_test["sum_check_subtotal"].reset_index().drop(columns = ["index"])
df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_perf["abs_diff"] = (df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_perf["predicted"] - df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_perf["actual"]).abs()
df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_perf["pct_diff"] = (df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_perf["abs_diff"]/df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_perf["actual"]*100).abs()
print("Average MAE in external test set is RM " + str(round(df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_perf["abs_diff"].mean(), 2)))
print("Average MAPE in external test set is " + str(round(df_sales_by_day_aeonbukitmertajam_incl_trans_type_takeaway_perf["pct_diff"].mean(), 2)) + "%")

Average MAE in internal test set is RM 463.96
Average MAPE in internal test set is 103.17%
Average MAE in external test set is RM 328.58
Average MAPE in external test set is 93.4%


In [126]:
#AEON BUKIT TINGGI: take-away
df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_ohe = pd.get_dummies(df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_econ, columns = ['day', 'Holiday'], dtype = float)
df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_ohe = df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_ohe.drop(columns = ["store_code", "casaname", "state", "trans_type_int", "trans_type", "sum_number_of_transactions", "sum_check_tax", "States"])

df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_ohe_train = df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_ohe[df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_ohe["date"] <= "2023-08-31"]
df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_ohe_test = df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_ohe[df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_ohe["date"] > "2023-08-31"]

X = list(df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_ohe_train.columns)
X.remove("sum_check_subtotal")
X.remove("date")

y = df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_ohe_train["sum_check_subtotal"]

X_df = df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_ohe_train[X]

x_train, x_test, y_train, y_test = train_test_split(X_df, y, test_size=0.15, random_state=42)

reg = GradientBoostingRegressor(random_state=0)

reg.fit(x_train, y_train)
y_pred = reg.predict(x_test)

#internal dataset test
df = pd.DataFrame()
df["y_pred"] = list(y_pred)
df["y_test"] = list(y_test)
df["difference"] = df["y_pred"] - df["y_test"]
df["abs_diff"] = df["difference"].abs()
df["pct_diff"] = (df["difference"]/df["y_test"]*100).abs()
print("Average MAE in internal test set is RM " + str(round(df["abs_diff"].mean(), 2)))
print("Average MAPE in internal test set is " + str(round(df["pct_diff"].mean(), 2)) + "%")

#external dataset test
df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_perf = pd.DataFrame(reg.predict(df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_ohe_test[X]), columns = ["predicted"])
df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_perf["actual"] = df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_ohe_test["sum_check_subtotal"].reset_index().drop(columns = ["index"])
df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_perf["abs_diff"] = (df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_perf["predicted"] - df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_perf["actual"]).abs()
df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_perf["pct_diff"] = (df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_perf["abs_diff"]/df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_perf["actual"]*100).abs()
print("Average MAE in external test set is RM " + str(round(df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_perf["abs_diff"].mean(), 2)))
print("Average MAPE in external test set is " + str(round(df_sales_by_day_aeonbukittinggi_incl_trans_type_takeaway_perf["pct_diff"].mean(), 2)) + "%")

Average MAE in internal test set is RM 396.77
Average MAPE in internal test set is 80.59%
Average MAE in external test set is RM 535.87
Average MAPE in external test set is 94.7%
